In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options

options = Options()
# options.add_argument('--headless')
driver = webdriver.Edge(options=options)

driver.get("https://skycoach.gg/")

# Click the "Choose a game" button to open the menu
# choose_game_btn = driver.find_element(By.CSS_SELECTOR, ".header__menu-button button button--action button--gradient-violet")
choose_game_btn = driver.find_element(By.CSS_SELECTOR, ".header__menu-button")
choose_game_btn.click()

import time
time.sleep(2)  # Adjust if needed

menu_content = driver.find_element(By.CLASS_NAME, "menu__wrapper").get_attribute('innerHTML')
print(menu_content)

driver.quit()


<div data-v-013755fa="" class="menu"><div data-v-013755fa="" class="menu__container container"><div data-v-013755fa="" class="menu__header"><!----> <div data-v-013755fa="" class="menu__logo-container"><img data-v-013755fa="" src="/images/logos/rebranded/logo.svg" alt="Skycoach" class="menu__logo"></div> <button data-v-f2586174="" data-v-013755fa="" class="menu__header-close-button button button--action"><svg data-v-039e196d="" data-v-013755fa="" aria-hidden="true" class="ui-icon" data-v-f2586174=""><use data-v-039e196d="" href="/images/icons/sprite.svg?v=14#x"></use></svg></button> <div data-v-04c8ddfb="" data-v-013755fa="" class="menu__search-mode-container menu__search-mode"><div data-v-281ef3c4="" data-v-04c8ddfb="" class="menu__search-mode-input input-container"><!----> <div data-v-281ef3c4="" class="input-background input-background--with-icon"><svg data-v-039e196d="" data-v-281ef3c4="" aria-hidden="true" class="ui-icon input-icon"><use data-v-039e196d="" href="/images/icons/sprit

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

soup = BeautifulSoup(menu_content, "html.parser")
games = []

for li in soup.find_all("li", class_="menu__game-section-item"):
    a_tag = li.find("a", class_="menu__game-section-item-link")
    if a_tag:
        name = a_tag.find("span").get_text(strip=True)
        href = a_tag.get("href")
        img_tag = a_tag.find("img", class_="menu__game-section-item-image")
        img_src = img_tag.get("src") if img_tag else None
        games.append({"Game Name": name, "Link": href, "Image": img_src})

df_games = pd.DataFrame(games)
print(df_games)


In [ ]:
df_games.to_csv("games.csv", index=False)


In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Load each game page with Selenium (Edge), wait for the sidebar to render,
then read driver.page_source and parse categories/subcategories with BeautifulSoup.

Input : games.csv  (must contain columns: "Game Name", "Link")
Output: One CSV per game under gamesTwo/<sanitized_game_name>_services.csv
"""

import os
import re
import time
import pandas as pd
from bs4 import BeautifulSoup

# Selenium (Edge)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ---------------------------------
# Config
# ---------------------------------
BASE_URL = "https://skycoach.gg"
INPUT_CSV = "games.csv"
OUTPUT_DIR = "gamesTwo"
WAIT_SECONDS = 20  # max wait for sidebar to appear

# ---------------------------------
# Helpers
# ---------------------------------
def sanitize_filename(name: str) -> str:
    # Replace colon and any invalid filename chars
    name = name.replace(":", "-")
    return re.sub(r'[\\/*?:"<>|]+', "-", name).strip()

def get_sidebar_html(driver) -> str:
    """
    Wait for the sidebar container to be present/visible and return full page_source.
    If the sidebar never appears, still return the final page_source after wait.
    """
    try:
        WebDriverWait(driver, WAIT_SECONDS).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".game-tags-sidebar__categories"))
        )
        # A tiny extra wait to let subnodes render after presence
        time.sleep(0.6)
    except Exception:
        # Sidebar didn't appear within timeout; we'll parse whatever is there.
        pass
    return driver.page_source

# ---------------------------------
# Main
# ---------------------------------
def main():
    # Read games list
    if not os.path.exists(INPUT_CSV):
        raise FileNotFoundError(f"{INPUT_CSV} not found")

    df = pd.read_csv(INPUT_CSV)

    # Validate columns
    needed_cols = {"Game Name", "Link"}
    missing = needed_cols - set(df.columns)
    if missing:
        raise ValueError(f"CSV missing required columns: {missing}")

    # Make output folder
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Selenium Edge setup
    edge_opts = EdgeOptions()
    edge_opts.page_load_strategy = "eager"
    # edge_opts.add_argument("--headless=new")  # uncomment for headless mode
    edge_opts.add_argument("--no-sandbox")
    edge_opts.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Edge(options=edge_opts)

    try:
        for _, row in df.iterrows():
            game_name_raw = str(row["Game Name"]).strip()
            game_link = str(row["Link"]).strip()

            if not game_link:
                print(f"[SKIP] Empty link for game: {game_name_raw}")
                continue

            game_name = sanitize_filename(game_name_raw)
            complete_game_link = BASE_URL + game_link

            print(f"\n=== Fetching: {game_name_raw} ===")
            print(f"URL: {complete_game_link}")

            # Load the page with Selenium and get fully rendered HTML
            driver.get(complete_game_link)
            html = get_sidebar_html(driver)

            # Parse with BeautifulSoup
            soup = BeautifulSoup(html, "html.parser")
            sidebar = soup.find("div", class_="game-tags-sidebar__categories")

            all_services = []
            if sidebar:
                # Iterate over main categories
                for cat in sidebar.find_all("div", class_="game-tags-sidebar__category", recursive=True):
                    head = cat.find("div", class_="game-tags-sidebar__category-head")

                    current_category_name = None
                    current_category_link = None

                    if head:
                        a_tag = head.find("a", class_="game-tags-sidebar__category-title")
                        if a_tag:
                            current_category_name = a_tag.get_text(strip=True)
                            current_category_link = a_tag.get("href")
                            all_services.append({
                                "Game Name": game_name_raw,
                                "Category": current_category_name,
                                "Subcategory": "",
                                "Link": BASE_URL + current_category_link if current_category_link else ""
                            })

                    # Subcategories (if any)
                    subcats = cat.find("div", class_="game-tags-sidebar__subcategories")
                    if subcats:
                        for sub_a in subcats.find_all("a", class_="game-tags-sidebar__subcategory"):
                            sub_name = sub_a.get_text(strip=True) if sub_a else ""
                            sub_link = sub_a.get("href") if sub_a else ""
                            all_services.append({
                                "Game Name": game_name_raw,
                                "Category": current_category_name or "",
                                "Subcategory": sub_name,
                                "Link": BASE_URL + sub_link if sub_link else ""
                            })
            else:
                print(f"[WARN] Sidebar not found for {game_name_raw}. Saving empty (or fallback) file.")

            # Save per-game CSV
            df_services = pd.DataFrame(all_services, columns=["Game Name", "Category", "Subcategory", "Link"])
            out_path = os.path.join(OUTPUT_DIR, f"{game_name.replace(' ', '_')}_services.csv")
            df_services.to_csv(out_path, index=False, encoding="utf-8")
            print(f"[OK] Saved: {out_path} ({len(df_services)} rows)")

    finally:
        driver.quit()
        print("\n[Done] Browser closed.")

if __name__ == "__main__":
    main()


There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache



=== Fetching: World of Warcraft ===
URL: https://skycoach.gg/wow-boost
[OK] Saved: gamesTwo/World_of_Warcraft_services.csv (26 rows)

=== Fetching: WoW Classic 20th Anniversary ===
URL: https://skycoach.gg/wow-classic-20th-anniversary-boost
[OK] Saved: gamesTwo/WoW_Classic_20th_Anniversary_services.csv (11 rows)

=== Fetching: Destiny 2 ===
URL: https://skycoach.gg/destiny-boost
[OK] Saved: gamesTwo/Destiny_2_services.csv (39 rows)

=== Fetching: Call of Duty ===
URL: https://skycoach.gg/call-of-duty-boost
[OK] Saved: gamesTwo/Call_of_Duty_services.csv (14 rows)

=== Fetching: Elden Ring Nightreign ===
URL: https://skycoach.gg/elden-ring-nightreign-boost
[OK] Saved: gamesTwo/Elden_Ring_Nightreign_services.csv (7 rows)

=== Fetching: 8 Ball Pool ===
URL: https://skycoach.gg/8-ball-pool-boost
[OK] Saved: gamesTwo/8_Ball_Pool_services.csv (1 rows)

=== Fetching: Albion Online ===
URL: https://skycoach.gg/albion-online-boost
[OK] Saved: gamesTwo/Albion_Online_services.csv (3 rows)

=== Fe

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: MicrosoftEdge=139.0.3405.111)
Stacktrace:
#0 0x61936487566e <unknown>
#1 0x61936432d0f6 <unknown>
#2 0x61936430537f <unknown>
#3 0x6193643abae5 <unknown>
#4 0x6193643c2f04 <unknown>
#5 0x6193643a2c83 <unknown>
#6 0x6193643708cb <unknown>
#7 0x6193643714b1 <unknown>
#8 0x619364845a11 <unknown>
#9 0x619364848c3b <unknown>
#10 0x61936484869b <unknown>
#11 0x6193648490f5 <unknown>
#12 0x619364832cb3 <unknown>
#13 0x6193648494c5 <unknown>
#14 0x61936481c395 <unknown>
#15 0x619364864a18 <unknown>
#16 0x619364864c3c <unknown>
#17 0x619364874b4c <unknown>
#18 0x7bbd53c94ac3 <unknown>


In [ ]:
import json

api_url = "https://skycoach.gg/api/shop/product/list/badge-products?badge_id=3&fields_data=compact"
response = requests.get(api_url)
data = response.json()
data_keys = data.keys()
print(data)
# Create a sample payload structure with "" values for each field
# sample_payload = {}
# for key in data_keys:
#     if isinstance(data[key], list) and data[key]:
#         sample_payload[key] = {field: "" for field in data[key][0].keys()}
#     else:
#         sample_payload[key] = None
# print(data)

# Save the sample payload to a JSON file
# with open("sample_payload.json", "w") as f:
#     json.dump(sample_payload, f, indent=4)

# save data in text file.txt
with open("data.txt", "w") as f:
    json.dump(data, f, indent=4)

# print(sample_payload)

for key in data_keys:
    print(f"Key: {key}")
    for item in data[key]:
        title = item.get("title", "No title")
        price = item.get("price", "No price")
        print(f"Title: {title}, Price: {price}")
# for item in data["2"]:
#     title = item.get("title", "No title")
#     price = item.get("price", "No price")
#     print(f"Title: {title}, Price: {price}")

In [ ]:
import os

games_files = set(os.listdir("games"))
gamesTwo_files = set(os.listdir("gamesTwo"))

# Find files present in gamesTwo but not in games
extra_in_gamesTwo = gamesTwo_files - games_files
# Find files present in games but not in gamesTwo
extra_in_games = games_files - gamesTwo_files

print("Extra files in gamesTwo:")
for fname in extra_in_gamesTwo:
    print(fname)

print("\nExtra files in games:")
for fname in extra_in_games:
    print(fname)

In [ ]:
# https://www.skycoach.gg/storage/uploads/game-assets/icons/8a9fa179ac.svg

# save this url svg in local directory /uploads
def save_svg_image(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
# Example usage
save_svg_image("https://www.skycoach.gg/storage/uploads/game-assets/icons/254932daa2.svg", "game_icon.svg")

In [ ]:
import pandas as pd
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.edge.options import Options
# import time
import os
# import requests
# from bs4 import BeautifulSoup
# options = Options()
# options.add_argument('--headless')
# driver = webdriver.Edge(options=options)
# /get all file names from gamesTwo directory
gamesTwo_files = os.listdir("gamesTwo")


# Loop through each file in the directory
for file_name in gamesTwo_files:
    file_name = "RBLX_services.csv"
    if file_name.endswith(".csv"):
        # Read the CSV file
        df = pd.read_csv(f"./gamesTwo/{file_name}")
        
        # Check if the 'Link' column exists
        if 'Link' in df.columns:
            # Loop through each row in the DataFrame
            for index, row in df.iterrows():
                link = row['Link']
                if link and link.startswith("https://skycoach.gg/"):
                    # extract data from link
                    print(f"Processing link: {link}")
                    # Here you can add your logic to process the link
                    page_source = requests.get(link).text
                    print(page_source)
                    """<div class="game-header" data-v-265d981e data-v-059b5164><div class="game-header__top-container" data-v-265d981e><h1 class="h1 sm:h1-mobile" data-v-265d981e>
      8 Ball Pool Coins
    </h1> <!----></div> <!----></div>
    
    <picture class="responsive-image offer-card__image" data-v-26f85f9a data-v-0d6f92b4><source media="(-webkit-min-device-pixel-ratio: 2)" type="" srcset="/storage/uploads/products/8-ball-pool-coins1717452870_picture_item.png" data-v-26f85f9a> <img src="/storage/uploads/products/8-ball-pool-coins1717452871_picture_item_small.png" data-fallback-img="/images/games/picture_item.png" class="responsive-image__img" data-v-26f85f9a></picture> 
 
       <div class="product-info-sections product-offer__description" data-v-3d474cdb data-v-059b5164><div class="product-info-section" data-v-3d474cdb><h2 class="product-info-section__title h2" data-v-3d474cdb>
      Description
    </h2> <div class="product-info-section__content" data-v-3d474cdb><div class="product-info-section__wrapper" data-v-3d474cdb><div class="product-info-section__html text-medium wysiwyg-content" data-v-3d474cdb><p><span>Buy cheap 8 Ball Pool Coins&nbsp;on our platform and be ahead of everyone else.</span></p>
<p><span>In 8 Ball Pool, players can use special props like more accurate aiming lines and power-enhancing items. By acquiring enough 8 Ball Pool coins, players can purchase these props to gain additional advantages and improve their gameplay. So, there is never enough of this valuable currency, and getting it is a very tiring and time-consuming process. So, why waste your precious time when you can quickly get our 8 Ball Pool Coins for sale?</span></p>
<p><span>Contact us now and get as many 8 Ball Pool Coins as you need with maximum safety and quick delivery!</span></p>
<ol>
<li><span>Selected amount of 8 Ball Pool Coins;</span></li>
<li><span>100% safety;</span></li>
<li><span>Complete safety and 24/7 support.</span></li>
</ol></div></div></div></div> <div class="product-info-section" data-v-3d474cdb><h2 class="product-info-section__title h2" data-v-3d474cdb>
      Requirements
    </h2> <div class="product-info-section__content" data-v-3d474cdb><div class="product-info-section__wrapper" data-v-3d474cdb><div class="product-info-section__html text-medium wysiwyg-content" data-v-3d474cdb><ul>
<li>Active account;</li>
<li>20+ level (Semi-pro Rank).</li>
</ul></div></div></div></div> <div class="product-info-section" data-v-3d474cdb><h2 class="product-info-section__title h2" data-v-3d474cdb>
      More info
    </h2> <div class="product-info-section__content" data-v-3d474cdb><div class="product-info-section__wrapper" data-v-3d474cdb><div class="product-info-section__html text-medium wysiwyg-content" data-v-3d474cdb><p><span>HOW DO WE DELIVER 8 BALL POOL COINS?</span></p>
<p>You can get cheap 8 Ball Pool Coins via the following completion method:</p>
<p><span>Comfort Trade:</span></p>
<ul>
<li><span>This is the quickest and the most reliable 8 Ball Pool Coins delivery method. Our pro-player will safely access your account and carry out your order. All security measures will be in constant use so that you can rest assured - your account is in good hands.&nbsp;</span></li>
</ul></div></div></div></div></div>



<a href="/8-ball-pool-boost" class="breadcrumbs__back-button router-link-active button button--secondary button--white-transparent-10" data-v-f2586174 data-v-a5397112><svg aria-hidden="true" class="ui-icon breadcrumbs__back-button-icon" data-v-039e196d data-v-a5397112><use href="/images/icons/sprite.svg?v=14#chevron-left" data-v-039e196d></use></svg> <span class="breadcrumbs__back-button-text" data-v-a5397112>Coins</span></a>
 
<div class="payment-summary__price" data-v-0ce13794><!----> <div class="payment-summary__price-row payment-summary__price-row-total" data-v-0ce13794><div class="payment-summary__label-column" data-v-0ce13794><span class="payment-summary__label-total text-medium" data-v-0ce13794>
          Total
        </span> <!----></div> <div class="payment-summary__price-column" data-v-0ce13794><!----> <span class="payment-summary__price-column-total h2" data-v-0ce13794>
          0,99 €
        </span></div></div></div>     
       
             """

    break

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
results = []
gamesTwo_files = list(os.listdir("gamesTwo"))
for file_name in gamesTwo_files:
  if file_name.endswith(".csv"):
    print(file_name)
    df = pd.read_csv(f"gamesTwo/{file_name}")
    category = file_name.replace("_services.csv", "").replace("_", " ")
    if 'Link' in df.columns:
      for index, row in df.iterrows():
        link = row['Link']
        if link and link.startswith("https://skycoach.gg/"):
          print(f"Processing link: {link}")
          page_source = requests.get(link).text
          soup = BeautifulSoup(page_source, "html.parser")
          print(soup)
          # Name
          name_tag = soup.find("div", class_="game-header")
          name = None
          if name_tag:
            h1 = name_tag.find("h1")
            if h1:
              name = h1.get_text(strip=True)
          
          # Description
          desc = None
          desc_section = soup.find("div", class_="product-info-section__html")
          if desc_section:
            desc = desc_section.get_text(separator="\n", strip=True)
          
          # Icon
          icon = None
          picture = soup.find("picture", class_="responsive-image")
          if picture:
            img = picture.find("img")
            if img and img.get("src"):
              icon = img["src"]
              if icon.startswith("/"):
                icon = "https://skycoach.gg" + icon
          
          # Price
          price = None
          price_span = soup.find("span", class_="payment-summary__price-column-total")
          if price_span:
            price = price_span.get_text(strip=True)
          
          # Category
          cat = row.get("Category", category)
          
          results.append({
            "Name": name,
            "Description": desc,
            "Icon": icon,
            "Price": price,
            "Category": cat,
            "Link": link
          })
    break

df_result = pd.DataFrame(results)
print(df_result)


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os
results = []
gamesTwo_files = set(os.listdir("gamesTwo"))
for file_name in gamesTwo_files:
  if file_name.endswith(".csv"):
    print(file_name)
    df = pd.read_csv(f"gamesTwo/{file_name}")
    category = file_name.replace("_services.csv", "").replace("_", " ")
    if 'Link' in df.columns:
      for index, row in df.iterrows():
        link = row['Link']
        if link and link.startswith("https://skycoach.gg/"):
          print(f"Processing link: https://skycoach.gg/roblox-boost{link}")
          page_source = requests.get("https://skycoach.gg/roblox-boost").text
          soup = BeautifulSoup(page_source, "html.parser")
          
        #   # Name
        #   name_tag = soup.find("div", class_="game-header")
        #   name = None
        #   if name_tag:
        #     h1 = name_tag.find("h1")
        #     if h1:
        #       name = h1.get_text(strip=True)
          
        #   # Description
        #   desc = None
        #   desc_section = soup.find("div", class_="product-info-section__html")
        #   if desc_section:
        #     desc = desc_section.get_text(separator="\n", strip=True)
          
        #   # Icon
        #   icon = None
        #   picture = soup.find("picture", class_="responsive-image")
        #   if picture:
        #     img = picture.find("img")
        #     if img and img.get("src"):
        #       icon = img["src"]
        #       if icon.startswith("/"):
        #         icon = "https://skycoach.gg" + icon
          
        #   # Price
        #   price = None
        #   price_span = soup.find("span", class_="payment-summary__price-column-total")
        #   if price_span:
        #     price = price_span.get_text(strip=True)
          
        #   # Category
        #   cat = row.get("Category", category)
          
        #   results.append({
        #     "Name": name,
        #     "Description": desc,
        #     "Icon": icon,
        #     "Price": price,
        #     "Category": cat,
        #     "Link": link
        #   })
    break

df_result = pd.DataFrame(results)
print(df_result)


In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os
results = []
gamesTwo_files = set(os.listdir("gamesTwo"))
for file_name in gamesTwo_files:
  if file_name.endswith(".csv"):
    print(file_name)
    df = pd.read_csv(f"gamesTwo/{file_name}")
    category = file_name.replace("_services.csv", "").replace("_", " ")
    if 'Link' in df.columns:
      for index, row in df.iterrows():
        print(row)
        subcat = row.get("Subcategory", "")
        if subcat:
          print(f"Processing subcategory: {subcat}")
        else:
          print(f"Processing category: {category}")
        link = row['Link']
        if link and link.startswith("https://skycoach.gg/"):
          print(f"Processing link: {link}")
          page_source = requests.get("https://skycoach.gg/path-of-exile-2-boost/products/exalted-orbs-6477").text
          soup = BeautifulSoup(page_source, "html.parser")
          # asides = soup.find_all("aside", class_="game-tags-sidebar game-main-page-sidebar")
          # soup2 = BeautifulSoup(str(asides), "html.parser")
          # hrefs = [a.get("href") for a in soup2.find_all("a") if a.get("href")]

          # Name
          name_tag = soup.find("div", class_="game-header")
          name = None
          if name_tag:
            h1 = name_tag.find("h1")
            if h1:
              name = h1.get_text(strip=True)
          
          # Description
          desc = None
          desc_section = soup.find("div", class_="product-info-section__html")
          if desc_section:
            desc = desc_section.get_text(separator="\n", strip=True)
          
          # Icon
          icon = None
          picture = soup.find("picture", class_="responsive-image")
          if picture:
            img = picture.find("img")
            if img and img.get("src"):
              icon = img["src"]
              if icon.startswith("/"):
                icon = "https://skycoach.gg" + icon
          
          # Price
          price = None
          price_span = soup.find("span", class_="payment-summary__price-column-total")
          if price_span:
            price = price_span.get_text(strip=True)
          
          # Category
          cat = row.get("Category", category)
          
          results.append({
            "Name": name,
            "Description": desc,
            "Icon": icon,
            "Price": price,
            "Category": cat,
            "Link": link
          })
    break


print(hrefs)


Monster_Hunter_Wilds_services.csv
Game Name                                 Monster Hunter Wilds
Category                                         Powerleveling
Link         https://skycoach.gg/monster-hunter-wilds-boost...
Name: 0, dtype: object
Processing category: Monster Hunter Wilds
Processing link: https://skycoach.gg/monster-hunter-wilds-boost/powerleveling
Game Name                                 Monster Hunter Wilds
Category                                               Gearing
Link         https://skycoach.gg/monster-hunter-wilds-boost...
Name: 1, dtype: object
Processing category: Monster Hunter Wilds
Processing link: https://skycoach.gg/monster-hunter-wilds-boost/gearing
Game Name                                 Monster Hunter Wilds
Category                                       Account Driving
Link         https://skycoach.gg/monster-hunter-wilds-boost...
Name: 2, dtype: object
Processing category: Monster Hunter Wilds
Processing link: https://skycoach.gg/monster-hunter-wi

In [4]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import os
options = Options()
# options.add_argument('--headless')  # Uncomment if you want to run in headless
count = 0

driver = webdriver.Edge(options=options)
results = []
gamesTwo_files = set(os.listdir("gamesTwo"))
for file_name in gamesTwo_files:
    if file_name.endswith(".csv"):
        existing_file = f"skycoach_results_{file_name}.csv"
        existing_file_folder = os.listdir("game_services")
        if existing_file in existing_file_folder:
            print(f"Skipping {file_name} as results already exist.")
            continue
        df = pd.read_csv(f"gamesTwo/{file_name}")
        category = file_name.replace("_services.csv", "").replace("_", " ")
        if 'Link' in df.columns:
            for index, row in df.iterrows():
                subcat = row.get("Subcategory", "")
                if subcat:
                    print(f"Processing subcategory: {subcat}")
                else:
                    print(f"Processing category: {category}")
                link = row['Link']
                if link and link.startswith("https://skycoach.gg/"):
                    print(f"Processing link: {link}")
                    # Use selenium driver to get page source
                    driver.get(link)
                    page_source = driver.page_source
                    soup = BeautifulSoup(page_source, "html.parser")
                    # Find all li elements inside the specified class and print their hrefs
                    container = soup.find("div", class_="card-list game-tag-page__container game-tag-page__products-list")
                    nestedlinks = []
                    if container:
                        li_tags = container.find_all("li")
                        for li in li_tags:
                            a_tag = li.find("a", href=True)
                            if a_tag:
                                href = a_tag["href"]
                                nestedlinks.append(href)
                    print(nestedlinks)
                    # Name
                    if nestedlinks:
                        for  nestedlink in nestedlinks:
                            if nestedlink.startswith("/"):
                                nestedlink = "https://skycoach.gg" + nestedlink
                        # Fetch the first link from nested links
                                print(f"Fetching details from: {nestedlink}")
                                driver.get(f'{nestedlink}')
                                time.sleep(2)
                                page_source = driver.page_source
                                soup = BeautifulSoup(page_source, "html.parser")
                                name_tag = soup.find("div", class_="game-header")
                                name = None
                                if name_tag:
                                    h1 = name_tag.find("h1")
                                    if h1:
                                        name = h1.get_text(strip=True)

                                # Description
                                desc = None
                                desc_section = soup.find("div", class_="product-info-section__html")
                                if desc_section:
                                    desc = desc_section.get_text(separator="\n", strip=True)

                                # Icon
                                icon = None
                                picture = soup.find("picture", class_="responsive-image")
                                if picture:
                                    img = picture.find("img")
                                    if img and img.get("src"):
                                        icon = img["src"]
                                        if icon.startswith("/"):
                                            icon = "https://skycoach.gg" + icon

                                # Price
                                price = None
                                price_span = soup.find("span", class_="payment-summary__price-column-total")
                                if price_span:
                                    price = price_span.get_text(strip=True)

                                # Category
                                cat = row.get("Category", category)

                                results.append({
                                    "Name": name,
                                    "Description": desc,
                                    "Icon": icon,
                                    "Price": price,
                                    "Category": cat,
                                    "Link": link
                                }) 
                                count += 1
                                print(count)
                
        
    df_result = pd.DataFrame(results)
    df_result.to_csv(f"skycoach_results_{file_name}.csv", index=False)
    df_result = []
print(df_result)


KeyboardInterrupt: 

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

options = Options()
options.page_load_strategy = 'eager'
# options.add_argument('--headless')  # Uncomment if you want to run in headless

def process_nestedlink(nestedlink, row, category):
    driver = webdriver.Edge(options=options)
    result = {}
    try:
        if nestedlink.startswith("/"):
            nestedlink = "https://skycoach.gg" + nestedlink
        driver.get(nestedlink)
        time.sleep(2)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")
        name_tag = soup.find("div", class_="game-header")
        name = None
        if name_tag:
            h1 = name_tag.find("h1")
            if h1:
                name = h1.get_text(strip=True)
        desc = None
        desc_section = soup.find("div", class_="product-info-section__html")
        if desc_section:
            desc = desc_section.get_text(separator="\n", strip=True)
        icon = None
        picture = soup.find("picture", class_="responsive-image")
        if picture:
            img = picture.findAll("img")
            # (old code, not needed for icon extraction)
        price = None
        price_span = soup.find("span", class_="payment-summary__price-column-total")
        if price_span:
            price = price_span.get_text(strip=True)
        cat = row.get("Category", category)

        # Extract game character image
        game_char_image = None
        image_container = soup.find("div", class_="offer-card__image-container")
        if image_container:
            picture_tag = image_container.find("picture", class_="responsive-image offer-card__image")
            if picture_tag:
                # Try to get src from <source> first, then <img>
                src = None
                source_tag = picture_tag.find("source")
                img_tag = picture_tag.find("img")
                if source_tag and source_tag.has_attr("srcset"):
                    src = source_tag["srcset"]
                elif img_tag and img_tag.has_attr("src"):
                    src = img_tag["src"]
                if src and src.startswith("/"):
                    src = "https://skycoach.gg" + src
                game_char_image = src

        result = {
            "Name": name,
            "Description": desc,
            "Icon": icon,
            "Price": price,
            "Category": cat,
            "Link": row['Link'],
            "GameCharImage": game_char_image
        }
    finally:
        driver.quit()
    return result


count = 0
results = []
gamesTwo_files = set(os.listdir("gamesTwo"))
for file_name in gamesTwo_files:
    if file_name.endswith(".csv"):
        existing_file = f"skycoach_results_{file_name}.csv"
        existing_file_folder = os.listdir("game_services")
        if existing_file in existing_file_folder:
            print(f"Skipping {file_name} as results already exist.")
            continue
        df = pd.read_csv(f"gamesTwo/{file_name}")
        category = file_name.replace("_services.csv", "").replace("_", " ")
        if 'Link' in df.columns:
            with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust max_workers as needed
                futures = []
                for index, row in df.iterrows():
                    subcat = row.get("Subcategory", "")
                    if subcat:
                        print(f"Processing subcategory: {subcat}")
                    else:
                        print(f"Processing category: {category}")
                    link = row['Link']
                    if link and link.startswith("https://skycoach.gg/"):
                        print(f"Processing link: {link}")
                        # Use a temporary driver to get nested links
                        driver = webdriver.Edge(options=options)
                        driver.get(link)
                        page_source = driver.page_source
                        soup = BeautifulSoup(page_source, "html.parser")
                        container = soup.find("div", class_="card-list game-tag-page__container game-tag-page__products-list")
                        nestedlinks = []
                        if container:
                            li_tags = container.find_all("li")
                            for li in li_tags:
                                a_tag = li.find("a", href=True)
                                if a_tag:
                                    href = a_tag["href"]
                                    nestedlinks.append(href)
                        driver.quit()
                        print(nestedlinks)
                        for nestedlink in nestedlinks:
                            futures.append(executor.submit(process_nestedlink, nestedlink, row, category))
                for future in as_completed(futures):
                    result = future.result()
                    if result:
                        results.append(result)
                        count += 1
                        print(count)
        df_result = pd.DataFrame(results)
        df_result.to_csv(f"skycoach_results_{file_name}.csv", index=False)
        results = []
print("Done.")


There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache


Processing subcategory: nan
Processing link: https://skycoach.gg/diablo-4-boost/season
['/diablo-4-boost/gold-coins', '/diablo-4-boost/products/powerleveling-2099', '/diablo-4-boost/products/paragon-leveling-2204', '/diablo-4-boost/products/diablo-4-belial-kill-7956', '/diablo-4-boost/products/runewords-5937', '/diablo-4-boost/products/lair-boss-kill-7961', '/diablo-4-boost/products/lair-boss-key-packs-7963', '/diablo-4-boost/products/leveling-bundles-8012', '/diablo-4-boost/products/horadric-jewels-8253', '/diablo-4-boost/products/dante-pet-8254', '/diablo-4-boost/products/horadric-phials-8256', '/diablo-4-boost/products/horadric-spell-8258', '/diablo-4-boost/products/horadric-knowledge-rank-8264', '/diablo-4-boost/products/diablo-4-unique-gear-boost-2400', '/diablo-4-boost/products/lair-boss-keys-7962', '/diablo-4-boost/products/mythic-runewords-bundle-7994', '/diablo-4-boost/products/the-pit-of-artificers-4592', '/diablo-4-boost/products/torment-difficulties-5948', '/diablo-4-boost/

There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache


[]
Processing subcategory: nan
Processing link: https://skycoach.gg/diablo-4-boost/bosses


There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
/tmp/ipykernel_373569/3319955046.py:40: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  img = picture.findAll("img")
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using dr

...

There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache


['/diablo-4-boost/products/powerleveling-2099', '/diablo-4-boost/products/paragon-leveling-2204', '/diablo-4-boost/products/leveling-bundles-8012', '/diablo-4-boost/products/torment-difficulties-5948', '/diablo-4-boost/products/codex-of-power-boost-4817', '/diablo-4-boost/products/diablo-4-season-journey-2456', '/diablo-4-boost/products/seasonal-campaign-boost-3965', '/diablo-4-boost/products/diablo-4-season-starter-pack-2457', '/diablo-4-boost/products/hardcore-powerleveling-2347', '/diablo-4-boost/products/diablo-4-story-mode-2206', '/diablo-4-boost/products/glyphs-leveling-2415', '/diablo-4-boost/products/diablo-4-the-heroic-roster-achievement-2739']
Processing subcategory: nan
Processing link: https://skycoach.gg/diablo-4-boost/gearing


There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache


['/diablo-4-boost/products/runewords-5937', '/diablo-4-boost/products/horadric-jewels-8253', '/diablo-4-boost/products/horadric-phials-8256', '/diablo-4-boost/products/horadric-spell-8258', '/diablo-4-boost/products/diablo-4-unique-gear-boost-2400', '/diablo-4-boost/products/mythic-runewords-bundle-7994', '/diablo-4-boost/products/mythic-unique-gear-5833', '/diablo-4-boost/products/codex-of-power-boost-4817', '/diablo-4-boost/products/infernal-compass-5490', '/diablo-4-boost/products/diablo-4-legendary-items-2373', '/diablo-4-boost/products/helltide-farming-4805', '/diablo-4-boost/products/build-check-7245', '/diablo-4-boost/products/diablo-4-gems-farm-2422', '/diablo-4-boost/products/heir-of-perdition-5787', '/diablo-4-boost/products/shroud-of-false-death-5790', '/diablo-4-boost/products/nesekem-the-herald-5791', '/diablo-4-boost/products/bis-ancestral-gear-2414', '/diablo-4-boost/products/resplendent-sparks-farm-4139', '/diablo-4-boost/products/ugly-bastard-helm-5825', '/diablo-4-boo

There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in th

In [2]:
import csv
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options
import time
from datetime import datetime

def parse_price_modifier(price_text):
    """Extract numeric price value from text like '+6,43 €' or 'Free'"""
    if not price_text or price_text.lower() in ['free', 'basic']:
        return 0.00
    
    # Remove currency symbols and extract number
    price_clean = re.sub(r'[^\d,.-]', '', price_text)
    price_clean = price_clean.replace(',', '.')
    
    try:
        return float(price_clean)
    except ValueError:
        return 0.00

def clear_csv_file(filename='service_options.csv'):
    """Clear the CSV file and create a new one with just headers"""
    fieldnames = [
        'option_id', 'service_id', 'parent_option_id', 'option_type', 'option_name',
        'option_label', 'option_value', 'price_modifier', 'min_value', 'max_value',
        'default_value', 'is_required', 'display_order', 'is_active', 'created_at', 'updated_at'
    ]
    
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
    
    print(f"Cleared CSV file: {filename}")

def get_next_option_id(filename='service_options.csv'):
    """Get the next available option_id from existing CSV file"""
    import os
    if not os.path.exists(filename):
        return 1
    
    try:
        with open(filename, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            max_id = 0
            for row in reader:
                try:
                    option_id = int(row['option_id'])
                    max_id = max(max_id, option_id)
                except (ValueError, KeyError):
                    continue
            return max_id + 1
    except Exception:
        return 1

def get_next_service_id(filename='service_options.csv'):
    """Get the next available service_id from existing CSV file"""
    import os
    if not os.path.exists(filename):
        return 1
    
    try:
        with open(filename, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            max_id = 0
            for row in reader:
                try:
                    service_id = int(row['service_id'])
                    max_id = max(max_id, service_id)
                except (ValueError, KeyError):
                    continue
            return max_id + 1
    except Exception:
        return 1

def extract_service_options(html_content, service_id=None, start_option_id=None):
    """Extract service options from HTML content"""
    soup = BeautifulSoup(html_content, 'html.parser')
    options = []
    
    # Get service ID - auto-increment if not provided
    if service_id is None:
        service_id = get_next_service_id()
    
    # Get starting option ID
    if start_option_id is None:
        option_id_counter = get_next_option_id()
    else:
        option_id_counter = start_option_id
        
    display_order = 1
    
    # Find the main options container
    options_container = soup.find('div', class_='product-detail-calculator__options')
    if not options_container:
        return options
    
    option_groups = options_container.find_all('div', class_='option-group')
    
    for group in option_groups:
        product_option = group.find('div', class_='product-option')
        if not product_option:
            continue
            
        # Get option label from header
        option_head = product_option.find('div', class_='product-option__head')
        option_label = ""
        if option_head:
            label_div = option_head.find('div', class_='product-option__label')
            if label_div:
                option_label = label_div.get_text(strip=True).replace(':', '')
        
        # Check for range/slider input
        range_cluster = product_option.find('div', class_='product-option-cluster-range')
        if range_cluster:
            # Handle range/slider options
            input_containers = range_cluster.find_all('div', class_='input-container')
            
            for idx, container in enumerate(input_containers):
                label_div = container.find('div', class_='label')
                input_tag = container.find('input')
                
                if label_div and input_tag:
                    input_label = label_div.get_text(strip=True)
                    default_val = input_tag.get('value', '')
                    
                    # Extract min/max from range scale if available
                    min_val, max_val = None, None
                    range_container = range_cluster.find('div', class_='range-container')
                    if range_container:
                        scale_items = range_container.find_all('div', class_='range__scale-item')
                        if scale_items:
                            try:
                                min_val = int(scale_items[0].get_text(strip=True))
                                max_val = int(scale_items[-1].get_text(strip=True))
                            except ValueError:
                                pass
                    
                    options.append({
                        'option_id': option_id_counter,
                        'service_id': service_id,
                        'parent_option_id': None,
                        'option_type': 'slider',
                        'option_name': f"{option_label.lower().replace(' ', '_')}_{input_label.lower().replace(' ', '_')}",
                        'option_label': input_label,
                        'option_value': default_val,
                        'price_modifier': 0.00,
                        'min_value': min_val,
                        'max_value': max_val,
                        'default_value': default_val,
                        'is_required': 1,
                        'display_order': display_order,
                        'is_active': 1,
                        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    option_id_counter += 1
                    display_order += 1
        
        # Check for dropdown/select
        select_cluster = product_option.find('div', class_='product-option-cluster-select')
        if select_cluster:
            select_tag = select_cluster.find('select')
            if select_tag:
                parent_id = option_id_counter
                
                # Create parent option for dropdown
                options.append({
                    'option_id': parent_id,
                    'service_id': service_id,
                    'parent_option_id': None,
                    'option_type': 'dropdown',
                    'option_name': option_label.lower().replace(' ', '_').replace(':', ''),
                    'option_label': option_label,
                    'option_value': None,
                    'price_modifier': 0.00,
                    'min_value': None,
                    'max_value': None,
                    'default_value': None,
                    'is_required': 1,
                    'display_order': display_order,
                    'is_active': 1,
                    'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                })
                option_id_counter += 1
                display_order += 1
                
                # Add dropdown options
                dropdown_options = select_tag.find_all('option')
                for opt in dropdown_options:
                    option_text = opt.get_text(strip=True)
                    option_val = opt.get('value', '')
                    
                    options.append({
                        'option_id': option_id_counter,
                        'service_id': service_id,
                        'parent_option_id': parent_id,
                        'option_type': 'dropdown',
                        'option_name': f"{option_label.lower().replace(' ', '_').replace(':', '')}_{option_text.lower().replace(' ', '_')}",
                        'option_label': option_text,
                        'option_value': option_val,
                        'price_modifier': 0.00,
                        'min_value': None,
                        'max_value': None,
                        'default_value': option_val if opt.get('selected') else None,
                        'is_required': 0,
                        'display_order': display_order,
                        'is_active': 1,
                        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    option_id_counter += 1
                    display_order += 1
        
        # Check for radio buttons
        radio_cluster = product_option.find('div', class_='product-option-cluster-radios')
        if radio_cluster:
            parent_id = option_id_counter
            
            # Create parent option for radio group
            options.append({
                'option_id': parent_id,
                'service_id': service_id,
                'parent_option_id': None,
                'option_type': 'radio',
                'option_name': option_label.lower().replace(' ', '_').replace(':', ''),
                'option_label': option_label,
                'option_value': None,
                'price_modifier': 0.00,
                'min_value': None,
                'max_value': None,
                'default_value': None,
                'is_required': 1,
                'display_order': display_order,
                'is_active': 1,
                'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })
            option_id_counter += 1
            display_order += 1
            
            # Add radio options
            radio_options = radio_cluster.find_all('div', class_='radio-option')
            for radio_opt in radio_options:
                input_tag = radio_opt.find('input', {'type': 'radio'})
                label_span = radio_opt.find('span', class_='radio-check__label')
                price_div = radio_opt.find('div', class_='radio-option__price')
                
                if input_tag and label_span:
                    # Get the label text more carefully - it might have nested elements
                    label_text_parts = []
                    for text_node in label_span.stripped_strings:
                        if text_node.strip() and not text_node.strip().startswith('+') and not text_node.strip().lower() in ['free']:
                            label_text_parts.append(text_node.strip())
                            break  # Take only the first meaningful text
                    
                    option_text = label_text_parts[0] if label_text_parts else label_span.get_text(strip=True).split('\n')[0].strip()
                    option_val = input_tag.get('value', '')
                    is_checked = input_tag.get('checked') is not None
                    
                    price_modifier = 0.00
                    if price_div:
                        price_text = price_div.get_text(strip=True)
                        price_modifier = parse_price_modifier(price_text)
                    
                    options.append({
                        'option_id': option_id_counter,
                        'service_id': service_id,
                        'parent_option_id': parent_id,
                        'option_type': 'radio',
                        'option_name': f"{option_label.lower().replace(' ', '_').replace(':', '')}_{option_text.lower().replace(' ', '_')}",
                        'option_label': option_text,
                        'option_value': option_val,
                        'price_modifier': price_modifier,
                        'min_value': None,
                        'max_value': None,
                        'default_value': option_val if is_checked else None,
                        'is_required': 0,
                        'display_order': display_order,
                        'is_active': 1,
                        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    option_id_counter += 1
                    display_order += 1
        
        # Check for checkboxes
        checkbox_cluster = product_option.find('div', class_='product-option-cluster-checkboxes')
        if checkbox_cluster:
            parent_id = option_id_counter
            
            # Create parent option for checkbox group
            options.append({
                'option_id': parent_id,
                'service_id': service_id,
                'parent_option_id': None,
                'option_type': 'checkbox',
                'option_name': option_label.lower().replace(' ', '_').replace(':', ''),
                'option_label': option_label,
                'option_value': None,
                'price_modifier': 0.00,
                'min_value': None,
                'max_value': None,
                'default_value': None,
                'is_required': 0,
                'display_order': display_order,
                'is_active': 1,
                'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })
            option_id_counter += 1
            display_order += 1
            
            # Add checkbox options
            checkbox_options = checkbox_cluster.find_all('div', class_='checkbox-option')
            for checkbox_opt in checkbox_options:
                input_tag = checkbox_opt.find('input', {'type': 'checkbox'})
                label_span = checkbox_opt.find('span', class_='radio-check__label')
                price_div = checkbox_opt.find('div', class_='checkbox-option__price')
                
                if input_tag and label_span:
                    option_text = label_span.get_text(strip=True)
                    option_val = input_tag.get('value', '')
                    is_checked = input_tag.get('checked') is not None
                    
                    price_modifier = 0.00
                    if price_div:
                        price_text = price_div.get_text(strip=True)
                        price_modifier = parse_price_modifier(price_text)
                    
                    options.append({
                        'option_id': option_id_counter,
                        'service_id': service_id,
                        'parent_option_id': parent_id,
                        'option_type': 'checkbox',
                        'option_name': f"{option_label.lower().replace(' ', '_').replace(':', '')}_{option_text.lower().replace(' ', '_').replace('%', 'percent')}",
                        'option_label': option_text,
                        'option_value': option_val,
                        'price_modifier': price_modifier,
                        'min_value': None,
                        'max_value': None,
                        'default_value': option_val if is_checked else None,
                        'is_required': 0,
                        'display_order': display_order,
                        'is_active': 1,
                        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    option_id_counter += 1
                    display_order += 1
    
    return options

def save_options_to_csv(options, filename='service_options.csv', append_mode=True):
    """Save options to CSV file"""
    if not options:
        print("No options to save")
        return
    
    fieldnames = [
        'option_id', 'service_id', 'parent_option_id', 'option_type', 'option_name',
        'option_label', 'option_value', 'price_modifier', 'min_value', 'max_value',
        'default_value', 'is_required', 'display_order', 'is_active', 'created_at', 'updated_at'
    ]
    
    import os
    file_exists = os.path.exists(filename)
    
    if append_mode and file_exists:
        # Append mode: add to existing file without header
        with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerows(options)
        print(f"Appended {len(options)} options to {filename}")
    else:
        # Write mode: create new file or overwrite with header
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(options)
        print(f"Saved {len(options)} options to {filename}")

def scrape_multiple_services(urls, clear_csv_first=False):
    """
    Scrape multiple service URLs with auto-incrementing service IDs
    
    Args:
        urls: List of URLs to scrape
        clear_csv_first: If True, clears the CSV file before starting
    
    Returns:
        List of tuples: [(service_id, options_count, url), ...]
    """
    if clear_csv_first:
        clear_csv_file()
    
    results = []
    
    for i, url in enumerate(urls):
        print(f"\\n=== Processing URL {i+1}/{len(urls)} ===")
        options = scrape_service_options(url, append_to_csv=True)
        
        if options:
            service_id = options[0]['service_id']
            results.append((service_id, len(options), url))
        else:
            results.append((None, 0, url))
        
        # Small delay between requests
        time.sleep(1)
    
    print(f"\\n=== Summary ===")
    for service_id, count, url in results:
        if service_id:
            print(f"Service {service_id}: {count} options from {url}")
        else:
            print(f"Failed to extract options from {url}")
    
    return results

def scrape_service_options(url, service_id=None, append_to_csv=True):
    """Main function to scrape service options from a URL"""
    options = webdriver.EdgeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    driver = webdriver.Edge(options=options)
    
    # Get service ID - auto-increment if not provided
    if service_id is None:
        service_id = get_next_service_id()
    
    try:
        if url.startswith("/"):
            url = "https://skycoach.gg" + url
        
        print(f"Scraping options from: {url} (Service ID: {service_id})")
        driver.get(url)
        time.sleep(3)  # Wait for page to load
        
        page_source = driver.page_source
        options_data = extract_service_options(page_source, service_id)
        
        if options_data:
            save_options_to_csv(options_data, append_mode=append_to_csv)
            print(f"Successfully extracted {len(options_data)} options for service {service_id}")
            return options_data
        else:
            print("No options found on the page")
            return []
            
    except Exception as e:
        print(f"Error scraping options: {str(e)}")
        return []
    finally:
        driver.quit()


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

options = Options()
options.page_load_strategy = 'eager'
# options.add_argument('--headless')  # Uncomment if you want to run in headless

def process_nestedlink(nestedlink, row, category):
    driver = webdriver.Edge(options=options)
    result = {}
    try:
        if nestedlink.startswith("/"):
            nestedlink = "https://skycoach.gg" + nestedlink
        print(nestedlink)
        driver.get(nestedlink)
        time.sleep(2)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")
        name_tag = soup.find("div", class_="game-header")
        name = None
        if name_tag:
            h1 = name_tag.find("h1")
            if h1:
                name = h1.get_text(strip=True)
        desc = None
        desc_section = soup.find("div", class_="product-info-section__html")
        if desc_section:
            desc = desc_section.get_text(separator="\n", strip=True)
        icon = None
        picture = soup.find("picture", class_="responsive-image")
        if picture:
            img = picture.findAll("img")
            # (old code, not needed for icon extraction)
        price = None
        price_span = soup.find("span", class_="payment-summary__price-column-total")
        if price_span:
            price = price_span.get_text(strip=True)
        cat = row.get("Category", category)

        # Extract game character image
        game_char_image = None
        image_container = soup.find("div", class_="offer-card__image-container")
        if image_container:
            picture_tag = image_container.find("picture", class_="responsive-image offer-card__image")
            if picture_tag:
                # Try to get src from <source> first, then <img>
                src = None
                source_tag = picture_tag.find("source")
                img_tag = picture_tag.find("img")
                if source_tag and source_tag.has_attr("srcset"):
                    src = source_tag["srcset"]
                elif img_tag and img_tag.has_attr("src"):
                    src = img_tag["src"]
                if src and src.startswith("/"):
                    src = "https://skycoach.gg" + src
                game_char_image = src

        result = {
            "Name": name,
            "Description": desc,
            "Icon": icon,
            "Price": price,
            "Category": cat,
            "Link": row['Link'],
            "GameCharImage": game_char_image
        }
    finally:
        driver.quit()
    return result


count = 0
results = []
gamesTwo_files = set(os.listdir("gamesTwo"))
for file_name in gamesTwo_files:
    if file_name.endswith(".csv"):
        existing_file = f"skycoach_results_{file_name}.csv"
        existing_file_folder = os.listdir("game_services")
        if existing_file in existing_file_folder:
            print(f"Skipping {file_name} as results already exist.")
            continue
        df = pd.read_csv(f"gamesTwo/{file_name}")
        category = file_name.replace("_services.csv", "").replace("_", " ")
        if 'Link' in df.columns:
            with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust max_workers as needed
                futures = []
                for index, row in df.iterrows():
                    subcat = row.get("Subcategory", "")
                    if subcat:
                        print(f"Processing subcategory: {subcat}")
                    else:
                        print(f"Processing category: {category}")
                    link = row['Link']
                    if link and link.startswith("https://skycoach.gg/"):
                        print(f"Processing link: {link}")
                        # Use a temporary driver to get nested links
                        driver = webdriver.Edge(options=options)
                        driver.get(link)
                        page_source = driver.page_source
                        soup = BeautifulSoup(page_source, "html.parser")
                        container = soup.find("div", class_="card-list game-tag-page__container game-tag-page__products-list")
                        nestedlinks = []
                        if container:
                            li_tags = container.find_all("li")
                            for li in li_tags:
                                a_tag = li.find("a", href=True)
                                if a_tag:
                                    href = a_tag["href"]
                                    nestedlinks.append(href)
                        driver.quit()
                        print(nestedlinks)
                        for nestedlink in nestedlinks:
                            futures.append(executor.submit(scrape_service_options, nestedlink))
                for future in as_completed(futures):
                    result = future.result()
                    if result:
                        results.append(result)
                        count += 1
                        print(count)
        df_result = pd.DataFrame(results)
        df_result.to_csv(f"skycoach_results_{file_name}.csv", index=False)
        results = []
print("Done.")


Processing subcategory: nan
Processing link: https://skycoach.gg/diablo-4-boost/season


There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache


['/diablo-4-boost/gold-coins', '/diablo-4-boost/products/powerleveling-2099', '/diablo-4-boost/products/paragon-leveling-2204', '/diablo-4-boost/products/diablo-4-belial-kill-7956', '/diablo-4-boost/products/runewords-5937', '/diablo-4-boost/products/lair-boss-kill-7961', '/diablo-4-boost/products/lair-boss-key-packs-7963', '/diablo-4-boost/products/leveling-bundles-8012', '/diablo-4-boost/products/horadric-jewels-8253', '/diablo-4-boost/products/dante-pet-8254', '/diablo-4-boost/products/horadric-phials-8256', '/diablo-4-boost/products/horadric-spell-8258', '/diablo-4-boost/products/horadric-knowledge-rank-8264', '/diablo-4-boost/products/diablo-4-unique-gear-boost-2400', '/diablo-4-boost/products/lair-boss-keys-7962', '/diablo-4-boost/products/mythic-runewords-bundle-7994', '/diablo-4-boost/products/the-pit-of-artificers-4592', '/diablo-4-boost/products/torment-difficulties-5948', '/diablo-4-boost/products/boss-packs-bundle-6063', '/diablo-4-boost/products/mythic-unique-gear-5833']


NameError: name 'scrape_service_options' is not defined

In [1]:
# --- Imports & setup ---
import os
import re
import csv
import time
import threading
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options as EdgeOptions

BASE_URL = "https://skycoach.gg"
INPUT_DIR = "gamesTwo"                  # where your *_services.csv live (e.g., Destiny_2_services.csv)
SERVICES_CSV = "services.csv"           # output for services schema
SERVICE_OPTIONS_CSV = "service_options.csv"  # output for options schema

# Selenium options
SEL_OPTS = EdgeOptions()
SEL_OPTS.page_load_strategy = "eager"
# SEL_OPTS.add_argument("--headless=new")
SEL_OPTS.add_argument("--no-sandbox")
SEL_OPTS.add_argument("--disable-dev-shm-usage")

# Thread safety for file writes and ID allocation
services_lock = threading.Lock()
options_lock = threading.Lock()
id_lock = threading.Lock()

# --- Helpers ---
def normalize_url(href: str) -> str:
    if not href:
        return href
    href = href.strip()
    if href.startswith("/"):
        return BASE_URL + href
    return href

def clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def parse_currency_to_decimal(txt: str) -> str:
    """
    Return a string like '1234.56' suitable for CSV/DB decimal fields.
    Handles symbols, thousand separators and commas as decimals.
    Returns '' if not parseable.
    """
    if not txt:
        return ""
    t = txt.strip().lower()
    if t in ["free", "basic"]:
        return "0.00"
    # keep digits, commas, dots, minus
    t = re.sub(r"[^\d,.\-]", "", t)
    # if there are both comma and dot, assume comma is thousands -> remove commas
    if "," in t and "." in t:
        t = t.replace(",", "")
    else:
        # if only comma, treat as decimal separator
        if "," in t and "." not in t:
            t = t.replace(",", ".")
    try:
        return f"{float(t):.2f}"
    except Exception:
        return ""

def ensure_services_csv():
    if not os.path.exists(SERVICES_CSV):
        with open(SERVICES_CSV, "w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            # schema columns + extra game_name for mapping later
            w.writerow([
                "service_id", "game_id", "name", "description",
                "price_per_unit", "sale_price", "icon_url", "category",
                "game_name"  # extra helper column, keep or drop during import
            ])

def ensure_options_csv():
    if not os.path.exists(SERVICE_OPTIONS_CSV):
        with open(SERVICE_OPTIONS_CSV, "w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            w.writerow([
                'option_id', 'service_id', 'parent_option_id', 'option_type', 'option_name',
                'option_label', 'option_value', 'price_modifier', 'min_value', 'max_value',
                'default_value', 'is_required', 'display_order', 'is_active', 'created_at', 'updated_at'
            ])

def get_next_service_id_from_services_csv() -> int:
    ensure_services_csv()
    with services_lock, open(SERVICES_CSV, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        max_id = 0
        for row in reader:
            try:
                sid = int(row["service_id"])
                if sid > max_id:
                    max_id = sid
            except Exception:
                continue
    return max_id + 1

def get_next_option_id_from_options_csv() -> int:
    ensure_options_csv()
    with options_lock, open(SERVICE_OPTIONS_CSV, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        max_id = 0
        for row in reader:
            try:
                oid = int(row["option_id"])
                if oid > max_id:
                    max_id = oid
            except Exception:
                continue
    return max_id + 1

# --- Option extraction (from your 2nd cell, adapted to be pure-HTML based) ---
def parse_price_modifier(price_text):
    if not price_text or price_text.lower() in ['free', 'basic']:
        return 0.00
    num = parse_currency_to_decimal(price_text)
    try:
        return float(num) if num else 0.00
    except Exception:
        return 0.00

def extract_service_options_from_html(html_content, fixed_service_id=None, start_option_id=None):
    soup = BeautifulSoup(html_content, 'html.parser')
    options = []

    service_id = fixed_service_id if fixed_service_id is not None else 0  # must be provided to keep referential integrity

    option_id_counter = start_option_id if start_option_id is not None else get_next_option_id_from_options_csv()
    display_order = 1

    options_container = soup.find('div', class_='product-detail-calculator__options')
    if not options_container:
        return options

    option_groups = options_container.find_all('div', class_='option-group')

    for group in option_groups:
        product_option = group.find('div', class_='product-option')
        if not product_option:
            continue

        # Get option label
        option_head = product_option.find('div', class_='product-option__head')
        option_label = ""
        if option_head:
            label_div = option_head.find('div', class_='product-option__label')
            if label_div:
                option_label = clean_text(label_div.get_text()).replace(':', '')

        # Slider / Range
        range_cluster = product_option.find('div', class_='product-option-cluster-range')
        if range_cluster:
            input_containers = range_cluster.find_all('div', class_='input-container')
            min_val, max_val = None, None
            range_container = range_cluster.find('div', class_='range-container')
            if range_container:
                scale_items = range_container.find_all('div', class_='range__scale-item')
                if scale_items:
                    try:
                        min_val = int(clean_text(scale_items[0].get_text()))
                        max_val = int(clean_text(scale_items[-1].get_text()))
                    except Exception:
                        pass

            for container in input_containers:
                label_div = container.find('div', class_='label')
                input_tag = container.find('input')

                if label_div and input_tag:
                    input_label = clean_text(label_div.get_text())
                    default_val = input_tag.get('value', '')

                    options.append({
                        'option_id': option_id_counter,
                        'service_id': service_id,
                        'parent_option_id': None,
                        'option_type': 'slider',
                        'option_name': f"{option_label.lower().replace(' ', '_')}_{input_label.lower().replace(' ', '_')}",
                        'option_label': input_label,
                        'option_value': default_val,
                        'price_modifier': 0.00,
                        'min_value': min_val,
                        'max_value': max_val,
                        'default_value': default_val,
                        'is_required': 1,
                        'display_order': display_order,
                        'is_active': 1,
                        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    option_id_counter += 1
                    display_order += 1

        # Dropdown
        select_cluster = product_option.find('div', class_='product-option-cluster-select')
        if select_cluster:
            select_tag = select_cluster.find('select')
            if select_tag:
                parent_id = option_id_counter
                options.append({
                    'option_id': parent_id,
                    'service_id': service_id,
                    'parent_option_id': None,
                    'option_type': 'dropdown',
                    'option_name': option_label.lower().replace(' ', '_').replace(':', ''),
                    'option_label': option_label,
                    'option_value': None,
                    'price_modifier': 0.00,
                    'min_value': None,
                    'max_value': None,
                    'default_value': None,
                    'is_required': 1,
                    'display_order': display_order,
                    'is_active': 1,
                    'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                })
                option_id_counter += 1
                display_order += 1

                for opt in select_tag.find_all('option'):
                    option_text = clean_text(opt.get_text())
                    option_val = opt.get('value', '')
                    options.append({
                        'option_id': option_id_counter,
                        'service_id': service_id,
                        'parent_option_id': parent_id,
                        'option_type': 'dropdown',
                        'option_name': f"{option_label.lower().replace(' ', '_').replace(':', '')}_{option_text.lower().replace(' ', '_')}",
                        'option_label': option_text,
                        'option_value': option_val,
                        'price_modifier': 0.00,
                        'min_value': None,
                        'max_value': None,
                        'default_value': option_val if opt.get('selected') else None,
                        'is_required': 0,
                        'display_order': display_order,
                        'is_active': 1,
                        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    option_id_counter += 1
                    display_order += 1

        # Radios
        radio_cluster = product_option.find('div', class_='product-option-cluster-radios')
        if radio_cluster:
            parent_id = option_id_counter
            options.append({
                'option_id': parent_id,
                'service_id': service_id,
                'parent_option_id': None,
                'option_type': 'radio',
                'option_name': option_label.lower().replace(' ', '_').replace(':', ''),
                'option_label': option_label,
                'option_value': None,
                'price_modifier': 0.00,
                'min_value': None,
                'max_value': None,
                'default_value': None,
                'is_required': 1,
                'display_order': display_order,
                'is_active': 1,
                'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })
            option_id_counter += 1
            display_order += 1

            radio_options = radio_cluster.find_all('div', class_='radio-option')
            for ro in radio_options:
                input_tag = ro.find('input', {'type': 'radio'})
                label_span = ro.find('span', class_='radio-check__label')
                price_div = ro.find('div', class_='radio-option__price')

                if input_tag and label_span:
                    # conservative label extraction
                    label_text = ""
                    for t in label_span.stripped_strings:
                        t = t.strip()
                        if t and not t.startswith('+') and t.lower() != 'free':
                            label_text = t
                            break
                    label_text = label_text or clean_text(label_span.get_text()).split('\n')[0].strip()
                    option_val = input_tag.get('value', '')
                    is_checked = input_tag.get('checked') is not None
                    price_modifier = parse_price_modifier(price_div.get_text(strip=True)) if price_div else 0.00

                    options.append({
                        'option_id': option_id_counter,
                        'service_id': service_id,
                        'parent_option_id': parent_id,
                        'option_type': 'radio',
                        'option_name': f"{option_label.lower().replace(' ', '_').replace(':', '')}_{label_text.lower().replace(' ', '_')}",
                        'option_label': label_text,
                        'option_value': option_val,
                        'price_modifier': price_modifier,
                        'min_value': None,
                        'max_value': None,
                        'default_value': option_val if is_checked else None,
                        'is_required': 0,
                        'display_order': display_order,
                        'is_active': 1,
                        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    option_id_counter += 1
                    display_order += 1

        # Checkboxes
        checkbox_cluster = product_option.find('div', class_='product-option-cluster-checkboxes')
        if checkbox_cluster:
            parent_id = option_id_counter
            options.append({
                'option_id': parent_id,
                'service_id': service_id,
                'parent_option_id': None,
                'option_type': 'checkbox',
                'option_name': option_label.lower().replace(' ', '_').replace(':', ''),
                'option_label': option_label,
                'option_value': None,
                'price_modifier': 0.00,
                'min_value': None,
                'max_value': None,
                'default_value': None,
                'is_required': 0,
                'display_order': display_order,
                'is_active': 1,
                'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })
            option_id_counter += 1
            display_order += 1

            for co in checkbox_cluster.find_all('div', class_='checkbox-option'):
                input_tag = co.find('input', {'type': 'checkbox'})
                label_span = co.find('span', class_='radio-check__label')
                price_div = co.find('div', class_='checkbox-option__price')
                if input_tag and label_span:
                    option_text = clean_text(label_span.get_text())
                    option_val = input_tag.get('value', '')
                    is_checked = input_tag.get('checked') is not None
                    price_modifier = parse_price_modifier(price_div.get_text(strip=True)) if price_div else 0.00
                    options.append({
                        'option_id': option_id_counter,
                        'service_id': service_id,
                        'parent_option_id': parent_id,
                        'option_type': 'checkbox',
                        'option_name': f"{option_label.lower().replace(' ', '_').replace(':', '')}_{option_text.lower().replace(' ', '_').replace('%','percent')}",
                        'option_label': option_text,
                        'option_value': option_val,
                        'price_modifier': price_modifier,
                        'min_value': None,
                        'max_value': None,
                        'default_value': option_val if is_checked else None,
                        'is_required': 0,
                        'display_order': display_order,
                        'is_active': 1,
                        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    option_id_counter += 1
                    display_order += 1

    return options

def append_options(options):
    if not options:
        return 0
    ensure_options_csv()
    with options_lock, open(SERVICE_OPTIONS_CSV, "a", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        for o in options:
            w.writerow([
                o['option_id'], o['service_id'], o['parent_option_id'], o['option_type'], o['option_name'],
                o['option_label'], o['option_value'], f"{o['price_modifier']:.2f}", 
                o['min_value'], o['max_value'], o['default_value'], o['is_required'],
                o['display_order'], o['is_active'], o['created_at'], o['updated_at']
            ])
    return len(options)

# --- Service page extraction (from your 1st cell, merged & improved) ---
def extract_service_info_and_options(nested_url: str, category: str, game_name: str):
    """
    Scrape a single service page:
      - allocate a unique service_id
      - extract service fields
      - extract options linked to the same service_id
      - write service row and options rows
    """
    driver = webdriver.Edge(options=SEL_OPTS)
    try:
        url = normalize_url(nested_url)
        driver.get(url)
        time.sleep(2.0)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # Allocate service_id once for this page
        with id_lock:
            service_id = get_next_service_id_from_services_csv()

        # Name (from header)
        name = None
        name_tag = soup.find("div", class_="game-header")
        if name_tag:
            h1 = name_tag.find("h1")
            if h1:
                name = clean_text(h1.get_text())
        if not name:
            # fallback: page title h1
            h1 = soup.find("h1")
            if h1:
                name = clean_text(h1.get_text())

        # Description
        desc = None
        desc_section = soup.find("div", class_="product-info-section__html")
        if desc_section:
            desc = clean_text(desc_section.get_text(separator="\n"))

        # Icon / image (prefer og:image)
        icon_url = None
        og_img = soup.find("meta", property="og:image")
        if og_img and og_img.get("content"):
            icon_url = normalize_url(og_img["content"])
        if not icon_url:
            image_container = soup.find("div", class_="offer-card__image-container")
            if image_container:
                picture_tag = image_container.find("picture", class_="responsive-image offer-card__image")
                if picture_tag:
                    src = None
                    source_tag = picture_tag.find("source")
                    img_tag = picture_tag.find("img")
                    if source_tag and source_tag.has_attr("srcset"):
                        src = source_tag["srcset"].split()[0]
                    elif img_tag and img_tag.has_attr("src"):
                        src = img_tag["src"]
                    icon_url = normalize_url(src) if src else None

        # Price (displayed total)
        price_per_unit = ""
        sale_price = ""
        price_span = soup.find("span", class_="payment-summary__price-column-total")
        if price_span:
            price_per_unit = parse_currency_to_decimal(price_span.get_text(strip=True))

        # Write service row
        ensure_services_csv()
        with services_lock, open(SERVICES_CSV, "a", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            # game_id left blank; game_name included for later mapping
            w.writerow([
                service_id, "", name or "", desc or "", price_per_unit, sale_price,
                icon_url or "", category or "", game_name
            ])

        # Extract and append options (share same service_id)
        options = extract_service_options_from_html(html, fixed_service_id=service_id)
        appended = append_options(options)

        return {
            "service_id": service_id,
            "name": name,
            "options_count": appended,
            "url": url
        }

    except Exception as e:
        return {"error": str(e), "url": nested_url}
    finally:
        driver.quit()

# --- Crawl each game CSV, find nested service links, scrape them ---
def get_nested_links_from_listing(listing_url: str):
    driver = webdriver.Edge(options=SEL_OPTS)
    try:
        url = normalize_url(listing_url)
        driver.get(url)
        time.sleep(1.5)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        container = soup.find("div", class_="card-list game-tag-page__container game-tag-page__products-list")
        nested = []
        if container:
            for li in container.find_all("li"):
                a = li.find("a", href=True)
                if a:
                    nested.append(normalize_url(a["href"]))
        return nested
    except Exception:
        return []
    finally:
        driver.quit()

def derive_game_name_from_filename(file_name: str) -> str:
    # e.g., "Destiny_2_services.csv" -> "Destiny 2"
    base = file_name.replace("_services.csv", "").replace(".csv", "")
    return base.replace("_", " ").strip()

def process_game_file(file_name: str, max_workers: int = 4):
    path = os.path.join(INPUT_DIR, file_name)
    df = pd.read_csv(path)
    category = derive_game_name_from_filename(file_name)  # you can change this if category differs from game
    game_name = category  # using same; you can change if needed

    results = []
    futures = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for _, row in df.iterrows():
            link = str(row.get("Link", "")).strip()
            if not link or not link.startswith(BASE_URL):
                continue
            nested_links = get_nested_links_from_listing(link)
            for nurl in nested_links:
                futures.append(executor.submit(extract_service_info_and_options, nurl, category, game_name))

        for fut in as_completed(futures):
            results.append(fut.result())
    return results

def main():
    ensure_services_csv()
    ensure_options_csv()

    gamesTwo_files = set(os.listdir(INPUT_DIR))
    # skip if an output for this game already done? (original code had per-file outputs)
    # Here we always append to global CSVs; dedup is out-of-scope by design.

    for file_name in sorted(gamesTwo_files):
        if not file_name.endswith(".csv"):
            continue
        print(f"\n=== Processing game file: {file_name} ===")
        res = process_game_file(file_name, max_workers=4)
        ok = sum(1 for r in res if r and not r.get("error"))
        print(f"Done {file_name}: {ok} services scraped ({len(res)} attempts).")

if __name__ == "__main__":
    main()


There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache



=== Processing game file: Destiny_2_services.csv ===


There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in th

Done Destiny_2_services.csv: 659 services scraped (670 attempts).

=== Processing game file: Diablo_4_services.csv ===


There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in th

Done Diablo_4_services.csv: 174 services scraped (174 attempts).


In [3]:
# --- Imports & setup ---
import os
import re
import csv
import time
import threading
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options as EdgeOptions

BASE_URL = "https://skycoach.gg"
INPUT_DIR = "gamesTwo"                  # where your *_services.csv live (e.g., Destiny_2_services.csv)
SERVICES_CSV = "services.csv"           # output for services schema
SERVICE_OPTIONS_CSV = "service_options.csv"  # output for options schema

# Selenium options
SEL_OPTS = EdgeOptions()
SEL_OPTS.page_load_strategy = "eager"
# SEL_OPTS.add_argument("--headless=new")
SEL_OPTS.add_argument("--no-sandbox")
SEL_OPTS.add_argument("--disable-dev-shm-usage")

# Thread safety for file writes and ID allocation
services_lock = threading.Lock()
options_lock = threading.Lock()
id_lock = threading.Lock()

# --- Helpers ---
def normalize_url(href: str) -> str:
    if not href:
        return href
    href = href.strip()
    if href.startswith("/"):
        return BASE_URL + href
    return href

def clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def parse_currency_to_decimal(txt: str) -> str:
    """
    Return a string like '1234.56' suitable for CSV/DB decimal fields.
    Handles symbols, thousand separators and commas as decimals.
    Returns '' if not parseable.
    """
    if not txt:
        return ""
    t = txt.strip().lower()
    if t in ["free", "basic"]:
        return "0.00"
    # keep digits, commas, dots, minus
    t = re.sub(r"[^\d,.\-]", "", t)
    # if there are both comma and dot, assume comma is thousands -> remove commas
    if "," in t and "." in t:
        t = t.replace(",", "")
    else:
        # if only comma, treat as decimal separator
        if "," in t and "." not in t:
            t = t.replace(",", ".")
    try:
        return f"{float(t):.2f}"
    except Exception:
        return ""

def ensure_services_csv():
    if not os.path.exists(SERVICES_CSV):
        with open(SERVICES_CSV, "w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            # schema columns + extra game_name for mapping later
            w.writerow([
                "service_id", "game_id", "name", "description",
                "price_per_unit", "sale_price", "icon_url", "category",
                "game_name"  # extra helper column, keep or drop during import
            ])

def ensure_options_csv():
    if not os.path.exists(SERVICE_OPTIONS_CSV):
        with open(SERVICE_OPTIONS_CSV, "w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            w.writerow([
                'option_id', 'service_id', 'parent_option_id', 'option_type', 'option_name',
                'option_label', 'option_value', 'price_modifier', 'min_value', 'max_value',
                'default_value', 'is_required', 'display_order', 'is_active', 'created_at', 'updated_at'
            ])

def get_next_service_id_from_services_csv() -> int:
    ensure_services_csv()
    with services_lock, open(SERVICES_CSV, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        max_id = 0
        for row in reader:
            try:
                sid = int(row["service_id"])
                if sid > max_id:
                    max_id = sid
            except Exception:
                continue
    return max_id + 1

def get_next_option_id_from_options_csv() -> int:
    ensure_options_csv()
    with options_lock, open(SERVICE_OPTIONS_CSV, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        max_id = 0
        for row in reader:
            try:
                oid = int(row["option_id"])
                if oid > max_id:
                    max_id = oid
            except Exception:
                continue
    return max_id + 1

# --- Option extraction (from your 2nd cell, adapted to be pure-HTML based) ---
def parse_price_modifier(price_text):
    if not price_text or price_text.lower() in ['free', 'basic']:
        return 0.00
    num = parse_currency_to_decimal(price_text)
    try:
        return float(num) if num else 0.00
    except Exception:
        return 0.00

def extract_service_options_from_html(html_content, fixed_service_id=None, start_option_id=None):
    soup = BeautifulSoup(html_content, 'html.parser')
    options = []

    service_id = fixed_service_id if fixed_service_id is not None else 0  # must be provided to keep referential integrity

    option_id_counter = start_option_id if start_option_id is not None else get_next_option_id_from_options_csv()
    display_order = 1

    options_container = soup.find('div', class_='product-detail-calculator__options')
    if not options_container:
        return options

    option_groups = options_container.find_all('div', class_='option-group')

    for group in option_groups:
        product_option = group.find('div', class_='product-option')
        if not product_option:
            continue

        # Get option label
        option_head = product_option.find('div', class_='product-option__head')
        option_label = ""
        if option_head:
            label_div = option_head.find('div', class_='product-option__label')
            if label_div:
                option_label = clean_text(label_div.get_text()).replace(':', '')

        # Slider / Range
        range_cluster = product_option.find('div', class_='product-option-cluster-range')
        if range_cluster:
            input_containers = range_cluster.find_all('div', class_='input-container')
            min_val, max_val = None, None
            range_container = range_cluster.find('div', class_='range-container')
            if range_container:
                scale_items = range_container.find_all('div', class_='range__scale-item')
                if scale_items:
                    try:
                        min_val = int(clean_text(scale_items[0].get_text()))
                        max_val = int(clean_text(scale_items[-1].get_text()))
                    except Exception:
                        pass

            for container in input_containers:
                label_div = container.find('div', class_='label')
                input_tag = container.find('input')

                if label_div and input_tag:
                    input_label = clean_text(label_div.get_text())
                    default_val = input_tag.get('value', '')

                    options.append({
                        'option_id': option_id_counter,
                        'service_id': service_id,
                        'parent_option_id': None,
                        'option_type': 'slider',
                        'option_name': f"{option_label.lower().replace(' ', '_')}_{input_label.lower().replace(' ', '_')}",
                        'option_label': input_label,
                        'option_value': default_val,
                        'price_modifier': 0.00,
                        'min_value': min_val,
                        'max_value': max_val,
                        'default_value': default_val,
                        'is_required': 1,
                        'display_order': display_order,
                        'is_active': 1,
                        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    option_id_counter += 1
                    display_order += 1

        # Dropdown
        select_cluster = product_option.find('div', class_='product-option-cluster-select')
        if select_cluster:
            select_tag = select_cluster.find('select')
            if select_tag:
                parent_id = option_id_counter
                options.append({
                    'option_id': parent_id,
                    'service_id': service_id,
                    'parent_option_id': None,
                    'option_type': 'dropdown',
                    'option_name': option_label.lower().replace(' ', '_').replace(':', ''),
                    'option_label': option_label,
                    'option_value': None,
                    'price_modifier': 0.00,
                    'min_value': None,
                    'max_value': None,
                    'default_value': None,
                    'is_required': 1,
                    'display_order': display_order,
                    'is_active': 1,
                    'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                })
                option_id_counter += 1
                display_order += 1

                for opt in select_tag.find_all('option'):
                    option_text = clean_text(opt.get_text())
                    option_val = opt.get('value', '')
                    options.append({
                        'option_id': option_id_counter,
                        'service_id': service_id,
                        'parent_option_id': parent_id,
                        'option_type': 'dropdown',
                        'option_name': f"{option_label.lower().replace(' ', '_').replace(':', '')}_{option_text.lower().replace(' ', '_')}",
                        'option_label': option_text,
                        'option_value': option_val,
                        'price_modifier': 0.00,
                        'min_value': None,
                        'max_value': None,
                        'default_value': option_val if opt.get('selected') else None,
                        'is_required': 0,
                        'display_order': display_order,
                        'is_active': 1,
                        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    option_id_counter += 1
                    display_order += 1

        # Radios
        radio_cluster = product_option.find('div', class_='product-option-cluster-radios')
        if radio_cluster:
            parent_id = option_id_counter
            options.append({
                'option_id': parent_id,
                'service_id': service_id,
                'parent_option_id': None,
                'option_type': 'radio',
                'option_name': option_label.lower().replace(' ', '_').replace(':', ''),
                'option_label': option_label,
                'option_value': None,
                'price_modifier': 0.00,
                'min_value': None,
                'max_value': None,
                'default_value': None,
                'is_required': 1,
                'display_order': display_order,
                'is_active': 1,
                'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })
            option_id_counter += 1
            display_order += 1

            radio_options = radio_cluster.find_all('div', class_='radio-option')
            for ro in radio_options:
                input_tag = ro.find('input', {'type': 'radio'})
                label_span = ro.find('span', class_='radio-check__label')
                price_div = ro.find('div', class_='radio-option__price')

                if input_tag and label_span:
                    # conservative label extraction
                    label_text = ""
                    for t in label_span.stripped_strings:
                        t = t.strip()
                        if t and not t.startswith('+') and t.lower() != 'free':
                            label_text = t
                            break
                    label_text = label_text or clean_text(label_span.get_text()).split('\n')[0].strip()
                    option_val = input_tag.get('value', '')
                    is_checked = input_tag.get('checked') is not None
                    price_modifier = parse_price_modifier(price_div.get_text(strip=True)) if price_div else 0.00

                    options.append({
                        'option_id': option_id_counter,
                        'service_id': service_id,
                        'parent_option_id': parent_id,
                        'option_type': 'radio',
                        'option_name': f"{option_label.lower().replace(' ', '_').replace(':', '')}_{label_text.lower().replace(' ', '_')}",
                        'option_label': label_text,
                        'option_value': option_val,
                        'price_modifier': price_modifier,
                        'min_value': None,
                        'max_value': None,
                        'default_value': option_val if is_checked else None,
                        'is_required': 0,
                        'display_order': display_order,
                        'is_active': 1,
                        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    option_id_counter += 1
                    display_order += 1

        # Checkboxes
        checkbox_cluster = product_option.find('div', class_='product-option-cluster-checkboxes')
        if checkbox_cluster:
            parent_id = option_id_counter
            options.append({
                'option_id': parent_id,
                'service_id': service_id,
                'parent_option_id': None,
                'option_type': 'checkbox',
                'option_name': option_label.lower().replace(' ', '_').replace(':', ''),
                'option_label': option_label,
                'option_value': None,
                'price_modifier': 0.00,
                'min_value': None,
                'max_value': None,
                'default_value': None,
                'is_required': 0,
                'display_order': display_order,
                'is_active': 1,
                'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })
            option_id_counter += 1
            display_order += 1

            for co in checkbox_cluster.find_all('div', class_='checkbox-option'):
                input_tag = co.find('input', {'type': 'checkbox'})
                label_span = co.find('span', class_='radio-check__label')
                price_div = co.find('div', class_='checkbox-option__price')
                if input_tag and label_span:
                    option_text = clean_text(label_span.get_text())
                    option_val = input_tag.get('value', '')
                    is_checked = input_tag.get('checked') is not None
                    price_modifier = parse_price_modifier(price_div.get_text(strip=True)) if price_div else 0.00
                    options.append({
                        'option_id': option_id_counter,
                        'service_id': service_id,
                        'parent_option_id': parent_id,
                        'option_type': 'checkbox',
                        'option_name': f"{option_label.lower().replace(' ', '_').replace(':', '')}_{option_text.lower().replace(' ', '_').replace('%','percent')}",
                        'option_label': option_text,
                        'option_value': option_val,
                        'price_modifier': price_modifier,
                        'min_value': None,
                        'max_value': None,
                        'default_value': option_val if is_checked else None,
                        'is_required': 0,
                        'display_order': display_order,
                        'is_active': 1,
                        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    option_id_counter += 1
                    display_order += 1

    return options

def append_options(options):
    if not options:
        return 0
    ensure_options_csv()
    with options_lock, open(SERVICE_OPTIONS_CSV, "a", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        for o in options:
            w.writerow([
                o['option_id'], o['service_id'], o['parent_option_id'], o['option_type'], o['option_name'],
                o['option_label'], o['option_value'], f"{o['price_modifier']:.2f}", 
                o['min_value'], o['max_value'], o['default_value'], o['is_required'],
                o['display_order'], o['is_active'], o['created_at'], o['updated_at']
            ])
    return len(options)

# --- Service page extraction (from your 1st cell, merged & improved) ---
def extract_service_info_and_options(nested_url: str, category: str, game_name: str):
    """
    Scrape a single service page:
      - allocate a unique service_id
      - extract service fields
      - extract options linked to the same service_id
      - write service row and options rows
    """
    driver = webdriver.Edge(options=SEL_OPTS)
    try:
        url = normalize_url(nested_url)
        driver.get(url)
        time.sleep(2.0)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # Allocate service_id once for this page
        with id_lock:
            service_id = get_next_service_id_from_services_csv()

        # Name (from header)
        name = None
        name_tag = soup.find("div", class_="game-header")
        if name_tag:
            h1 = name_tag.find("h1")
            if h1:
                name = clean_text(h1.get_text())
        if not name:
            # fallback: page title h1
            h1 = soup.find("h1")
            if h1:
                name = clean_text(h1.get_text())

        # Description
        desc = None
        desc_section = soup.find("div", class_="product-info-section__html")
        if desc_section:
            desc = clean_text(desc_section.get_text(separator="\n"))

        # Icon / image (prefer og:image)
        icon_url = None
        og_img = soup.find("meta", property="og:image")
        if og_img and og_img.get("content"):
            icon_url = normalize_url(og_img["content"])
        if not icon_url:
            image_container = soup.find("div", class_="offer-card__image-container")
            if image_container:
                picture_tag = image_container.find("picture", class_="responsive-image offer-card__image")
                if picture_tag:
                    src = None
                    source_tag = picture_tag.find("source")
                    img_tag = picture_tag.find("img")
                    if source_tag and source_tag.has_attr("srcset"):
                        src = source_tag["srcset"].split()[0]
                    elif img_tag and img_tag.has_attr("src"):
                        src = img_tag["src"]
                    icon_url = normalize_url(src) if src else None

        # Price (displayed total)
        price_per_unit = ""
        sale_price = ""
        price_span = soup.find("span", class_="payment-summary__price-column-total")
        if price_span:
            price_per_unit = parse_currency_to_decimal(price_span.get_text(strip=True))

        # Write service row
        ensure_services_csv()
        with services_lock, open(SERVICES_CSV, "a", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            # game_id left blank; game_name included for later mapping
            w.writerow([
                service_id, "", name or "", desc or "", price_per_unit, sale_price,
                icon_url or "", category or "", game_name
            ])

        # Extract and append options (share same service_id)
        options = extract_service_options_from_html(html, fixed_service_id=service_id)
        appended = append_options(options)

        return {
            "service_id": service_id,
            "name": name,
            "options_count": appended,
            "url": url
        }

    except Exception as e:
        return {"error": str(e), "url": nested_url}
    finally:
        driver.quit()

# --- Crawl each game CSV, find nested service links, scrape them ---
def get_nested_links_from_listing(listing_url: str):
    driver = webdriver.Edge(options=SEL_OPTS)
    try:
        url = normalize_url(listing_url)
        driver.get(url)
        time.sleep(1.5)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        container = soup.find("div", class_="card-list game-tag-page__container game-tag-page__products-list")
        nested = []
        if container:
            for li in container.find_all("li"):
                a = li.find("a", href=True)
                if a:
                    nested.append(normalize_url(a["href"]))
        return nested
    except Exception:
        return []
    finally:
        driver.quit()

def derive_game_name_from_filename(file_name: str) -> str:
    # e.g., "Destiny_2_services.csv" -> "Destiny 2"
    base = file_name.replace("_services.csv", "").replace(".csv", "")
    return base.replace("_", " ").strip()

def process_game_file(file_name: str, max_workers: int = 4):
    path = os.path.join(INPUT_DIR, file_name)
    df = pd.read_csv(path)
    category = derive_game_name_from_filename(file_name)  # you can change this if category differs from game
    game_name = category  # using same; you can change if needed

    results = []
    futures = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for _, row in df.iterrows():
            link = str(row.get("Link", "")).strip()
            if not link or not link.startswith(BASE_URL):
                continue
            nested_links = get_nested_links_from_listing(link)
            for nurl in nested_links:
                futures.append(executor.submit(extract_service_info_and_options, nurl, category, game_name))

        for fut in as_completed(futures):
            results.append(fut.result())
    return results

def main():
    ensure_services_csv()
    ensure_options_csv()

    gamesTwo_files = set(os.listdir(INPUT_DIR))
    # skip if an output for this game already done? (original code had per-file outputs)
    # Here we always append to global CSVs; dedup is out-of-scope by design.

    for file_name in sorted(gamesTwo_files):
        if not file_name.endswith(".csv"):
            continue
        print(f"\n=== Processing game file: {file_name} ===")
        res = process_game_file(file_name, max_workers=4)
        ok = sum(1 for r in res if r and not r.get("error"))
        print(f"Done {file_name}: {ok} services scraped ({len(res)} attempts).")

if __name__ == "__main__":
    ensure_services_csv()
    ensure_options_csv()

    test_url = "https://skycoach.gg/destiny-boost/products/opaque-hourglass-8613"
    category = "Destiny 2"   # or whatever you want as category
    game_name = "Destiny 2"

    result = extract_service_info_and_options(test_url, category, game_name)

    print("\n=== Single link test result ===")
    print(result)


There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache



=== Single link test result ===
{'service_id': 897, 'name': 'Opaque Hourglass Crossbow', 'options_count': 8, 'url': 'https://skycoach.gg/destiny-boost/products/opaque-hourglass-8613'}


In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import math
import pandas as pd
import mysql.connector
from mysql.connector import errorcode

# ---------------------------
# Config
# ---------------------------
DB_CONFIG = {
    "host": "localhost",
    "port": 3306,
    "user": "root",
    "password": "",
    "database": "boostgg",
}

SERVICES_CSV = "services.csv"
SERVICE_OPTIONS_CSV = "service_options.csv"

# Map game_name from services.csv to an existing DB game_id
GAME_NAME_TO_ID = {
    "Destiny 2": 21,
    "Diablo 4": 22,
    # add more...
}

# Behavior toggles
REUSE_EXISTING_SERVICE_BY_NAME = True
FILL_PARENT_VALUE_WHEN_NULL = True  # set parent option_value = option_name for checkbox/radio/dropdown
DEDUPE_PARENTS_BY_OPTION_NAME = True
RESEQUENCE_DISPLAY_ORDER = True
STRIP_LEADING_UNDERSCORE_FOR_SLIDER = False  # set True only if your UI hides names like "_current_level"

# ---------------------------
# Skip/normalize helpers
# ---------------------------
ERROR_PAGE_MARKERS = [
    "hmmm… can't reach this page",
    "can't reach this page",
    "this site can’t be reached",
    "problem loading page",
]

def looks_like_error_page(text: str) -> bool:
    if not text:
        return False
    t = text.strip().lower()
    return any(marker in t for marker in ERROR_PAGE_MARKERS)

def normalize_game_name(raw: str) -> str:
    """
    Handle cases like 'Destiny 2,Destiny 2' -> 'Destiny 2'
    - Split on commas, take first non-empty token, strip spaces.
    """
    if not raw:
        return ""
    parts = [p.strip() for p in str(raw).split(",")]
    for p in parts:
        if p:
            return p
    return ""

def norm_game_id(game_name):
    # Normalize and map strictly; fallback to case-insensitive equality
    norm = normalize_game_name(game_name)
    if norm in GAME_NAME_TO_ID:
        return GAME_NAME_TO_ID[norm]
    for k, gid in GAME_NAME_TO_ID.items():
        if k.lower().strip() == norm.lower().strip():
            return gid
    return None

# ---------------------------
# Type/NULL helpers
# ---------------------------
def is_nullish(v):
    return v is None or (isinstance(v, float) and math.isnan(v)) or (isinstance(v, str) and v.strip() == "")

def to_nullable_decimal(s):
    if is_nullish(s):
        return None
    try:
        return float(str(s))
    except Exception:
        return None

def to_nullable_int(v):
    if is_nullish(v):
        return None
    try:
        return int(float(v))
    except Exception:
        return None

def to_nullable_str(v):
    if is_nullish(v):
        return None
    return str(v)

# ---------------------------
# Options normalization (before DB insert)
# ---------------------------
def _is_parent_row(row) -> bool:
    p = row.get("parent_option_id")
    return (p is None) or (str(p).strip() == "") or (str(p).lower() == "nan")

def _to_int_series(s: pd.Series):
    return pd.to_numeric(s, errors="coerce").fillna(0).astype(int)

def normalize_options_dataframe(df_opts: pd.DataFrame) -> pd.DataFrame:
    """
    Returns a cleaned copy of df_opts with:
      - filled parent option_value for group types
      - deduped parent groups by option_name (children re-parented)
      - resequenced display_order for parents and for children under each parent
      - optional strip of leading '_' in slider parent option_name
    """
    if df_opts.empty:
        return df_opts

    df = df_opts.copy()

    # Ensure all key cols exist as strings
    for col in ["option_id", "service_id", "parent_option_id", "option_type", "option_name",
                "option_label", "option_value", "price_modifier", "min_value", "max_value",
                "default_value", "is_required", "display_order", "is_active",
                "created_at", "updated_at"]:
        if col not in df.columns:
            df[col] = ""

    # Normalize parent_option_id empty/null to ""
    df["parent_option_id"] = df["parent_option_id"].astype(str)
    df.loc[df["parent_option_id"].str.strip().isin(["", "None", "nan", "NaN"]), "parent_option_id"] = ""

    # A) fill parent option_value when null (for group types)
    if FILL_PARENT_VALUE_WHEN_NULL:
        is_parent = df.apply(_is_parent_row, axis=1)
        is_group_parent = is_parent & df["option_type"].isin(["checkbox", "radio", "dropdown"])
        empty_val = df["option_value"].astype(str).str.strip().isin(["", "None", "nan", "NaN"])
        mask = is_group_parent & empty_val
        df.loc[mask, "option_value"] = df.loc[mask, "option_name"]

    # B) dedupe parents by option_name (keep first by display_order then option_id)
    if DEDUPE_PARENTS_BY_OPTION_NAME:
        parents = df[df.apply(_is_parent_row, axis=1)].copy()
        parents["option_id_int"] = _to_int_series(parents["option_id"])
        parents["display_order_int"] = _to_int_series(parents["display_order"])
        parents = parents.sort_values(["display_order_int", "option_id_int"], kind="stable")

        # keep first per option_name
        keepers = parents.drop_duplicates(subset=["option_name"], keep="first")
        kept_ids = set(keepers["option_id"].tolist())

        # find duplicates to drop
        dup_parents = parents[~parents["option_id"].isin(keepers["option_id"])]
        if not dup_parents.empty:
            # map duplicate parent option_id -> kept option_id per option_name
            name_to_keep = keepers.set_index("option_name")["option_id"].to_dict()
            dup_map = dup_parents.set_index("option_id")["option_name"].to_dict()
            # re-parent children of duplicates to the kept parent
            df["parent_option_id"] = df["parent_option_id"].replace({dup_id: name_to_keep[dup_name]
                                                                     for dup_id, dup_name in dup_map.items()
                                                                     if dup_name in name_to_keep})
            # drop duplicate parent rows
            df = df[~((df["option_id"].isin(dup_parents["option_id"])) & (df["parent_option_id"] == ""))]

    # C) resequence display_order
    if RESEQUENCE_DISPLAY_ORDER:
        # Parents
        parents = df[df.apply(_is_parent_row, axis=1)].copy()
        parents["option_id_int"] = _to_int_series(parents["option_id"])
        parents["display_order_int"] = _to_int_series(parents["display_order"])
        parents = parents.sort_values(["display_order_int", "option_id_int"], kind="stable")
        parents["new_display"] = range(1, len(parents) + 1)
        df.loc[parents.index, "display_order"] = parents["new_display"].astype(str)

        # Children per parent
        for parent_id in parents["option_id"].tolist():
            kids = df[df["parent_option_id"] == parent_id].copy()
            if kids.empty:
                continue
            kids["option_id_int"] = _to_int_series(kids["option_id"])
            kids["display_order_int"] = _to_int_series(kids["display_order"])
            kids = kids.sort_values(["display_order_int", "option_id_int"], kind="stable")
            kids["new_display"] = range(1, len(kids) + 1)
            df.loc[kids.index, "display_order"] = kids["new_display"].astype(str)

    # D) optionally strip leading '_' from slider parent option_name
    if STRIP_LEADING_UNDERSCORE_FOR_SLIDER:
        mask = df.apply(_is_parent_row, axis=1) & (df["option_type"] == "slider") & df["option_name"].str.startswith("_")
        df.loc[mask, "option_name"] = df.loc[mask, "option_name"].str[1:]

    return df

# ---------------------------
# DB insertion helpers
# ---------------------------
def insert_one_option(cur, db_service_id, orow, parent_db_id=None):
    """
    Insert a single option row, return new DB option_id.
    Expects columns from service_options.csv (post-normalization).
    """
    option_type = to_nullable_str(orow["option_type"])
    option_name = to_nullable_str(orow["option_name"])
    option_label = to_nullable_str(orow["option_label"])
    option_value = to_nullable_str(orow["option_value"])

    price_modifier = to_nullable_decimal(orow["price_modifier"])
    min_value = to_nullable_int(orow["min_value"])
    max_value = to_nullable_int(orow["max_value"])
    default_value = to_nullable_str(orow["default_value"])

    is_required = to_nullable_int(orow["is_required"])
    display_order = to_nullable_int(orow["display_order"])
    is_active = to_nullable_int(orow["is_active"])

    created_at = to_nullable_str(orow["created_at"])
    updated_at = to_nullable_str(orow["updated_at"])

    sql = """
        INSERT INTO service_options
        (service_id, parent_option_id, option_type, option_name, option_label, option_value, price_modifier,
         min_value, max_value, default_value, is_required, display_order, is_active, created_at, updated_at)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cur.execute(sql, (
        db_service_id, parent_db_id, option_type, option_name, option_label, option_value, price_modifier,
        min_value, max_value, default_value, is_required, display_order, is_active, created_at, updated_at
    ))
    return cur.lastrowid

# ---------------------------
# Main import routine
# ---------------------------
def main():
    # Load CSVs
    df_services = pd.read_csv(SERVICES_CSV, dtype=str, keep_default_na=False)
    df_options_all = pd.read_csv(SERVICE_OPTIONS_CSV, dtype=str, keep_default_na=False)

    # Validate columns
    required_service_cols = {"service_id", "name", "description", "price_per_unit", "sale_price", "icon_url", "category", "game_name"}
    missing = required_service_cols - set(df_services.columns)
    if missing:
        raise ValueError(f"services.csv missing columns: {missing}")

    required_option_cols = {
        "option_id", "service_id", "parent_option_id", "option_type", "option_name",
        "option_label", "option_value", "price_modifier", "min_value", "max_value",
        "default_value", "is_required", "display_order", "is_active", "created_at", "updated_at"
    }
    missing_opt = required_option_cols - set(df_options_all.columns)
    if missing_opt:
        raise ValueError(f"service_options.csv missing columns: {missing_opt}")

    # Connect DB
    cnx = mysql.connector.connect(**DB_CONFIG)
    cnx.autocommit = False
    cur = cnx.cursor()

    insert_service_sql = """
        INSERT INTO services (game_id, name, description, price_per_unit, sale_price, icon_url, category)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    select_service_sql = """
        SELECT service_id FROM services WHERE game_id = %s AND name = %s LIMIT 1
    """

    total_services = 0
    total_options = 0

    for _, srow in df_services.iterrows():
        csv_service_id = srow["service_id"]
        game_name_raw = srow["game_name"]

        # Map/normalize game_name -> game_id
        game_id = norm_game_id(game_name_raw)
        if not game_id:
            print(f"[SKIP] Unmapped or malformed game_name='{game_name_raw}' (csv_service_id={csv_service_id})")
            continue

        name = to_nullable_str(srow["name"])
        description = to_nullable_str(srow["description"])

        # Skip rows that look like browser error captures
        if looks_like_error_page(name) or looks_like_error_page(description):
            print(f"[SKIP] Looks like an error page captured (csv_service_id={csv_service_id}, name='{name}')")
            continue

        price_per_unit = to_nullable_decimal(srow["price_per_unit"])
        sale_price = to_nullable_decimal(srow["sale_price"])
        icon_url = to_nullable_str(srow["icon_url"])
        category = to_nullable_str(srow["category"])

        try:
            # Transaction per service (service + options)
            cnx.start_transaction()

            # Optionally reuse existing service by (game_id, name)
            db_service_id = None
            if REUSE_EXISTING_SERVICE_BY_NAME and name:
                cur.execute(select_service_sql, (game_id, name))
                row = cur.fetchone()
                if row:
                    db_service_id = int(row[0])

            # Insert service if not found
            if not db_service_id:
                cur.execute(
                    insert_service_sql,
                    (game_id, name, description, price_per_unit, sale_price, icon_url, category)
                )
                db_service_id = cur.lastrowid

            # Options for this service (match CSV service_id)
            df_opts_raw = df_options_all[df_options_all["service_id"] == str(csv_service_id)].copy()

            # Normalize the options for this service before inserting
            df_opts = normalize_options_dataframe(df_opts_raw)

            # Partition into parents and children (post-normalization)
            parents = df_opts[df_opts.apply(_is_parent_row, axis=1)].copy()
            children = df_opts[~df_opts.apply(_is_parent_row, axis=1)].copy()

            # Map csv option_id -> db option_id
            option_id_map = {}

            # Insert parents first (in display order)
            parents["option_id_int"] = pd.to_numeric(parents["option_id"], errors="coerce").fillna(0).astype(int)
            parents["display_order_int"] = pd.to_numeric(parents["display_order"], errors="coerce").fillna(0).astype(int)
            parents = parents.sort_values(["display_order_int", "option_id_int"], kind="stable")

            for _, orow in parents.iterrows():
                db_oid = insert_one_option(cur, db_service_id, orow, parent_db_id=None)
                option_id_map[str(orow["option_id"])] = db_oid
                total_options += 1

            # Insert children with mapped parent ids (in display order)
            children["option_id_int"] = pd.to_numeric(children["option_id"], errors="coerce").fillna(0).astype(int)
            children["display_order_int"] = pd.to_numeric(children["display_order"], errors="coerce").fillna(0).astype(int)
            children = children.sort_values(["parent_option_id", "display_order_int", "option_id_int"], kind="stable")

            for _, orow in children.iterrows():
                csv_parent = str(orow["parent_option_id"]).strip()
                parent_db_id = option_id_map.get(csv_parent)
                if not parent_db_id:
                    # Parent missing—warn and set NULL to avoid crash (should be rare after normalization)
                    print(f"[WARN] Missing parent mapping for csv_parent_option_id={csv_parent} "
                          f"(service csv_id={csv_service_id}, service name={name}); setting NULL parent.")
                    parent_db_id = None
                db_oid = insert_one_option(cur, db_service_id, orow, parent_db_id=parent_db_id)
                option_id_map[str(orow["option_id"])] = db_oid
                total_options += 1

            cnx.commit()
            total_services += 1
            print(f"[OK] Service '{name}' (game_id={game_id}) -> DB service_id={db_service_id}, "
                  f"options={len(df_opts)}")

        except Exception as e:
            cnx.rollback()
            print(f"[ROLLBACK] Service '{name}' (csv_service_id={csv_service_id}) failed: {e}")

    cur.close()
    cnx.close()
    print(f"\nDone. Imported services: {total_services}, options: {total_options}")

# ---------------------------
# Entrypoint
# ---------------------------
if __name__ == "__main__":
    try:
        main()
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("MySQL auth error: check user/password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print("MySQL error:", err)
    except Exception as e:
        print("Fatal error:", e)


[ROLLBACK] Service 'Opaque Hourglass Crossbow' (csv_service_id=1) failed: 1265 (01000): Data truncated for column 'option_type' at row 1

Done. Imported services: 0, options: 2


In [9]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import math
import pandas as pd
import mysql.connector
from mysql.connector import errorcode

# ---------------------------
# Config
# ---------------------------
DB_CONFIG = {
    "host": "localhost",
    "port": 3306,
    "user": "root",
    "password": "",
    "database": "boostgg",
}

SERVICES_CSV = "services.csv"
SERVICE_OPTIONS_CSV = "service_options.csv"

# Behavior
FORCE_GAME_ID = 21                  # always use 21, ignore CSV
REUSE_EXISTING_SERVICE_BY_NAME = True  # reuse service if (game_id, name) already exists

# ---------------------------
# Helpers
# ---------------------------
def is_nullish(v):
    return v is None or (isinstance(v, float) and math.isnan(v)) or (isinstance(v, str) and v.strip() == "")

def to_nullable_decimal(s):
    if is_nullish(s):
        return None
    try:
        return float(str(s))
    except Exception:
        return None

def to_nullable_int(v):
    if is_nullish(v):
        return None
    try:
        return int(float(v))
    except Exception:
        return None

def to_nullable_str(v):
    if is_nullish(v):
        return None
    return str(v)

def is_parent_row(orow) -> bool:
    p = orow.get("parent_option_id")
    return (p is None) or (str(p).strip() == "") or (str(p).lower() == "nan")

def clean_option_types(df: pd.DataFrame) -> pd.DataFrame:
    """Normalize option_type for insert:
       - remove *_value suffix
       - buttons/button -> radio
    """
    df = df.copy()
    df["option_type"] = df["option_type"].astype(str)
    df["option_type"] = df["option_type"].str.replace(r"_value$", "", regex=True)
    df["option_type"] = df["option_type"].replace({"buttons": "radio", "button": "radio"})
    return df

# ---------------------------
# DB insert helpers (RAW SQL)
# ---------------------------
INSERT_SERVICE_SQL = """
INSERT INTO services (game_id, name, description, price_per_unit, sale_price, icon_url, category)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

SELECT_SERVICE_SQL = """
SELECT service_id FROM services WHERE game_id = %s AND name = %s LIMIT 1
"""

INSERT_OPTION_SQL = """
INSERT INTO service_options
(service_id, parent_option_id, option_type, option_name, option_label, option_value,
 price_modifier, min_value, max_value, default_value, is_required, display_order,
 is_active, created_at, updated_at)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

def insert_service(cur, row):
    """Insert a single service (force game_id=21, ignore CSV PK) and return DB service_id.
       If REUSE_EXISTING_SERVICE_BY_NAME, tries to reuse existing by (game_id, name).
    """
    game_id = FORCE_GAME_ID
    name = to_nullable_str(row.get("name"))
    description = to_nullable_str(row.get("description"))
    price_per_unit = to_nullable_decimal(row.get("price_per_unit"))
    sale_price = to_nullable_decimal(row.get("sale_price"))
    icon_url = to_nullable_str(row.get("icon_url"))
    category = to_nullable_str(row.get("category"))

    if REUSE_EXISTING_SERVICE_BY_NAME and name:
        cur.execute(SELECT_SERVICE_SQL, (game_id, name))
        existing = cur.fetchone()
        if existing:
            return int(existing[0])

    cur.execute(
        INSERT_SERVICE_SQL,
        (game_id, name, description, price_per_unit, sale_price, icon_url, category)
    )
    return cur.lastrowid

def insert_one_option(cur, db_service_id, orow, parent_db_id=None):
    """Insert one option row and return new DB option_id."""
    option_type = to_nullable_str(orow.get("option_type"))
    option_name = to_nullable_str(orow.get("option_name"))
    option_label = to_nullable_str(orow.get("option_label"))
    option_value = to_nullable_str(orow.get("option_value"))

    price_modifier = to_nullable_decimal(orow.get("price_modifier"))
    min_value = to_nullable_int(orow.get("min_value"))
    max_value = to_nullable_int(orow.get("max_value"))
    default_value = to_nullable_str(orow.get("default_value"))

    is_required = to_nullable_int(orow.get("is_required"))
    display_order = to_nullable_int(orow.get("display_order"))
    is_active = to_nullable_int(orow.get("is_active"))

    created_at = to_nullable_str(orow.get("created_at"))
    updated_at = to_nullable_str(orow.get("updated_at"))

    cur.execute(INSERT_OPTION_SQL, (
        db_service_id, parent_db_id, option_type, option_name, option_label, option_value,
        price_modifier, min_value, max_value, default_value, is_required, display_order,
        is_active, created_at, updated_at
    ))
    return cur.lastrowid

# ---------------------------
# Main
# ---------------------------
def main():
    # Read CSVs
    df_services = pd.read_csv(SERVICES_CSV, dtype=str, keep_default_na=False)
    df_options_all = pd.read_csv(SERVICE_OPTIONS_CSV, dtype=str, keep_default_na=False)

    # Basic column checks
    needed_svc_cols = {"name", "description", "price_per_unit", "sale_price", "icon_url", "category", "game_id", "game_name"}
    missing = needed_svc_cols - set(df_services.columns)
    if missing:
        raise ValueError(f"services.csv missing columns: {missing}")

    needed_opt_cols = {
        "option_id","service_id","parent_option_id","option_type","option_name",
        "option_label","option_value","price_modifier","min_value","max_value",
        "default_value","is_required","display_order","is_active","created_at","updated_at"
    }
    miss_opt = needed_opt_cols - set(df_options_all.columns)
    if miss_opt:
        raise ValueError(f"service_options.csv missing columns: {miss_opt}")

    # Normalize option types (remove *_value, buttons/button -> radio)
    df_options_all = clean_option_types(df_options_all)

    # Connect DB
    cnx = mysql.connector.connect(**DB_CONFIG)
    cnx.autocommit = False
    cur = cnx.cursor()

    imported_services = 0
    imported_options = 0

    try:
        # If services.csv has many rows but no stable linkage to options (by csv service_id),
        # we only support 1 service in that case. Prefer including a service_id column in services.csv.
        has_csv_service_id = "service_id" in df_services.columns

        for _, srow in df_services.iterrows():
            cnx.start_transaction()
            # 1) Insert/reuse service, capture DB PK
            db_service_id = insert_service(cur, srow)

            # 2) Determine which options to use for this service
            if has_csv_service_id:
                csv_sid = str(srow.get("service_id", "")).strip()
                df_opts_raw = df_options_all[df_options_all["service_id"] == csv_sid].copy()
            else:
                # No service_id in services.csv — assume single service and take ALL options
                if len(df_services) > 1:
                    raise RuntimeError("Multiple services present but services.csv has no service_id column to match options.csv groups.")
                df_opts_raw = df_options_all.copy()

            # 3) Split into parents/children (based on parent_option_id empty)
            parents = df_opts_raw[df_opts_raw.apply(is_parent_row, axis=1)].copy()
            children = df_opts_raw[~df_opts_raw.apply(is_parent_row, axis=1)].copy()

            # stable sort: by display_order then option_id for parents; by parent then display_order for children
            parents["display_order_int"] = pd.to_numeric(parents["display_order"], errors="coerce").fillna(0).astype(int)
            parents["option_id_int"] = pd.to_numeric(parents["option_id"], errors="coerce").fillna(0).astype(int)
            parents = parents.sort_values(["display_order_int", "option_id_int"], kind="stable")

            children["parent_option_id_int"] = pd.to_numeric(children["parent_option_id"], errors="coerce").fillna(0).astype(int)
            children["display_order_int"] = pd.to_numeric(children["display_order"], errors="coerce").fillna(0).astype(int)
            children["option_id_int"] = pd.to_numeric(children["option_id"], errors="coerce").fillna(0).astype(int)
            children = children.sort_values(["parent_option_id_int", "display_order_int", "option_id_int"], kind="stable")

            # 4) Insert parents, track CSV option_id -> DB option_id
            id_map = {}  # csv option_id (str) -> db option_id (int)
            for _, prow in parents.iterrows():
                new_id = insert_one_option(cur, db_service_id, prow, parent_db_id=None)
                id_map[str(prow["option_id"])] = new_id
                imported_options += 1

            # 5) Insert children using mapped parent IDs
            for _, crow in children.iterrows():
                csv_parent = str(crow["parent_option_id"]).strip()
                parent_db_id = id_map.get(csv_parent)
                if not parent_db_id:
                    # parent missing — insert as top-level to avoid failure (or raise)
                    print(f"[WARN] Missing parent for csv option_id={crow.get('option_id')} (csv parent={csv_parent}); inserting with NULL parent.")
                    parent_db_id = None
                new_id = insert_one_option(cur, db_service_id, crow, parent_db_id=parent_db_id)
                id_map[str(crow["option_id"])] = new_id
                imported_options += 1

            cnx.commit()
            imported_services += 1
            print(f"[OK] Inserted/linked service '{srow.get('name')}' (DB id {db_service_id}) with {len(df_opts_raw)} options.")

    except Exception as e:
        cnx.rollback()
        print("[ROLLBACK] Error:", e)
        raise
    finally:
        cur.close()
        cnx.close()

    print(f"\nDone. Services inserted/linked: {imported_services}, options inserted: {imported_options}")

# ---------------------------
# Entrypoint
# ---------------------------
if __name__ == "__main__":
    try:
        main()
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("MySQL auth error: check user/password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print("MySQL error:", err)
    except Exception as e:
        print("Fatal error:", e)


[OK] Inserted/linked service 'The Desert Perpetual Raid' (DB id 2563) with 28 options.
[OK] Inserted/linked service 'Power Level (Light Level) Boost' (DB id 2564) with 4 options.
[OK] Inserted/linked service 'The Edge of Fate Campaign' (DB id 2565) with 78 options.
[OK] Inserted/linked service 'Kepler Exploration' (DB id 2566) with 23 options.
[OK] Inserted/linked service 'Graviton Spike' (DB id 2567) with 16 options.
[OK] Inserted/linked service 'Best Legendary Armor' (DB id 2568) with 20 options.
[OK] Inserted/linked service 'Flawless Desert Perpetual Raid' (DB id 2569) with 3 options.
[OK] Inserted/linked service 'Epic Desert Perpetual Raid' (DB id 2570) with 26 options.
[OK] Inserted/linked service 'Legendary Campaign + Power Cap' (DB id 2571) with 10 options.
[OK] Inserted/linked service 'Rewards Pass Boost' (DB id 2572) with 5 options.
[OK] Inserted/linked service 'Unstable Cores' (DB id 2573) with 8 options.
[OK] Inserted/linked service 'Soloist Ghost Shell' (DB id 2574) with 2 

In [7]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
dedupe_services_and_prune_options.py

Goal:
1) Remove duplicate rows in services.csv where BOTH 'name' and 'description' are identical
   (keep the first occurrence).
2) Remove rows from service_options.csv whose 'service_id' belongs to the removed services.

Usage:
  python dedupe_services_and_prune_options.py [services_csv] [service_options_csv]

Defaults:
  services_csv       = services.csv
  service_options_csv = service_options.csv
"""

import sys
import os
import shutil
from datetime import datetime
import pandas as pd

def backup_file(path: str) -> str:
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    backup_path = f"{path}.bak.{ts}"
    shutil.copy2(path, backup_path)
    return backup_path

def main():
    services_csv = "./services.csv"
    options_csv  = "./service_options.csv"

    if not os.path.exists(services_csv):
        raise FileNotFoundError(f"services CSV not found: {services_csv}")
    if not os.path.exists(options_csv):
        raise FileNotFoundError(f"service_options CSV not found: {options_csv}")

    # --- Load CSVs as strings, keep empty strings instead of NaN ---
    df_services = pd.read_csv(services_csv, dtype=str, keep_default_na=False)
    df_options  = pd.read_csv(options_csv,  dtype=str, keep_default_na=False)

    # Remember original column order
    svc_cols = df_services.columns.tolist()
    opt_cols = df_options.columns.tolist()

    # --- Validate required columns ---
    svc_required = {"service_id", "name", "description"}
    missing_svc = svc_required - set(df_services.columns)
    if missing_svc:
        raise ValueError(f"{services_csv} missing required columns: {missing_svc}")

    opt_required = {
        "option_id","service_id","parent_option_id","option_type","option_name",
        "option_label","option_value","price_modifier","min_value","max_value",
        "default_value","is_required","display_order","is_active","created_at","updated_at"
    }
    missing_opt = opt_required - set(df_options.columns)
    if missing_opt:
        raise ValueError(f"{options_csv} missing required columns: {missing_opt}")

    # --- Compute duplicates in services on BOTH name+description (keep first) ---
    before_services = len(df_services)
    dup_mask = df_services.duplicated(subset=["name", "description"], keep="first")
    removed_services_df = df_services[dup_mask].copy()
    kept_services_df    = df_services[~dup_mask].copy()
    after_services = len(kept_services_df)
    removed_count = before_services - after_services

    # IDs of removed services
    removed_service_ids = set(removed_services_df["service_id"].astype(str).tolist())

    # --- Filter options: drop any whose service_id is in removed_service_ids ---
    before_options = len(df_options)
    if removed_service_ids:
        df_options_clean = df_options[~df_options["service_id"].astype(str).isin(removed_service_ids)].copy()
    else:
        df_options_clean = df_options.copy()
    after_options = len(df_options_clean)
    pruned_options = before_options - after_options

    # --- Backups ---
    svc_backup = backup_file(services_csv)
    opt_backup = backup_file(options_csv)
    print(f"[BACKUP] {services_csv} -> {svc_backup}")
    print(f"[BACKUP] {options_csv}  -> {opt_backup}")

    # --- Save cleaned data back, preserving original column order ---
    kept_services_df[svc_cols].to_csv(services_csv, index=False, encoding="utf-8")
    df_options_clean[opt_cols].to_csv(options_csv, index=False, encoding="utf-8")

    # --- Summary ---
    print(f"[SERVICES] Input: {before_services} | Removed dupes: {removed_count} | Output: {after_services}")
    print(f"[OPTIONS ] Input: {before_options}  | Pruned by service_id: {pruned_options} | Output: {after_options}")

if __name__ == "__main__":
    main()


[BACKUP] ./services.csv -> ./services.csv.bak.20250824-134440
[BACKUP] ./service_options.csv  -> ./service_options.csv.bak.20250824-134440
[SERVICES] Input: 488 | Removed dupes: 128 | Output: 360
[OPTIONS ] Input: 6619  | Pruned by service_id: 1324 | Output: 5295


In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
find_bad_csv_rows.py

Report rows in a CSV whose number of fields != the header's number of fields.
This helps track errors like:
  ParserError: Expected 9 fields in line 21, saw 18

Usage:
  python find_bad_csv_rows.py services.csv
"""

import sys
import csv

def main():
    path = "./services.csv"

    with open(path, newline="", encoding="utf-8") as f:
        reader = csv.reader(f)
        try:
            header = next(reader)
        except StopIteration:
            print("[ERROR] Empty file.")
            return

        expected = len(header)
        print(f"[INFO] Header has {expected} fields: {header}")

        bad_count = 0
        # Line numbers: header is line 1, so data starts at line 2
        for lineno, row in enumerate(reader, start=2):
            try:
                actual = len(row)
            except csv.Error as e:
                # csv module parsing error (e.g., unbalanced quotes)
                print(f"[CSV ERROR] at line {lineno}: {e}")
                bad_count += 1
                continue

            if actual != expected:
                bad_count += 1
                # Print a short preview so you can spot what’s off
                preview = repr(row)[:200]
                print(f"[BAD] line {lineno}: expected {expected}, got {actual}. Row preview: {preview}")

        if bad_count == 0:
            print("[OK] No malformed lines found.")
        else:
            print(f"[DONE] Found {bad_count} malformed line(s).")

if __name__ == "__main__":
    main()


[INFO] Header has 9 fields: ['service_id', 'game_id', 'name', 'description', 'price_per_unit', 'sale_price', 'icon_url', 'category', 'game_name']
[BAD] line 21: expected 9, got 10. Row preview: ['20', '21', 'Precipial', "Buy Precipial Void Shotgun from Destiny 2 Edge of Fate expansion and enjoy the Skycoach’s professional boosting service. Select the desired weapon Tier, choose your god roll
[DONE] Found 1 malformed line(s).
